# Feature Engineering

## Imports

In [ ]:
import pandas as pd
import numpy as np
import asyncio
import nest_asyncio
import aiohttp
import time
import os
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.utils import resample
import umap
import scipy.sparse
from typing import List, Dict, Tuple, Any, Set, Optional
import inflect
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.ticker as mtick
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import json
import pickle
import hdbscan
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

## Loading Data

In [ ]:
df = pd.read_parquet('arxiv_cleaned.parquet', engine='pyarrow')

## Non-Text Features

### Number of Authors

In [ ]:
df['number_of_authors'] = df['authors_parsed'].apply(len)

# Drop the original column
df = df.drop(columns=['authors_parsed'])

### Journal - DEPRECATED -

Retriving Journal names from Crossref API by doi.

In [ ]:
# # --- Configuration Parameters ---
# CHUNK_SIZE = 100          # How many DOIs to process in each batch
# CONCURRENCY_LIMIT = 4     # Max simultaneous requests
# DELAY_PER_REQUEST = 0.01   # Seconds to wait BEFORE each request attempt (e.g., 0.3 = ~3 req/sec max)
# RETRY_INITIAL_DELAY = 5   # Seconds to wait after first 429 error
# MAX_RETRIES = 3           # Max attempts for a single DOI
# DELAY_BETWEEN_CHUNKS = 0.5 # Seconds to pause between processing chunks
# OUTPUT_FILE = 'doi_journal_results.csv' # File to save progress
# USER_AGENT = 'mailto:martina.esser@iu-study.org; purpose: Academic Research Mapping DOIs, Feature Engineering for a Study Project' 
# # -----------------------------

# async def fetch_journal_async(session, doi, max_retries=MAX_RETRIES, initial_delay=RETRY_INITIAL_DELAY):
#     """
#     Asynchronously fetches the journal name for a given DOI using the Crossref API.
#     Includes retry logic, pre-request delay, and respects configured parameters.
#     """
#     # --- Polite Pre-Request Delay ---
#     await asyncio.sleep(DELAY_PER_REQUEST)
#     # -------------------------------

#     if pd.isna(doi):
#         return doi, None

#     url = f"https://api.crossref.org/works/{doi}"
#     headers = {'User-Agent': USER_AGENT}

#     current_delay = initial_delay
#     for attempt in range(max_retries):
#         try:
#             async with session.get(url, headers=headers, timeout=30) as response:
#                 if response.status == 200:
#                     try:
#                         data = await response.json()
#                         journal_name = data.get('message', {}).get('container-title', [None])[0]
#                         return doi, journal_name
#                     except aiohttp.ContentTypeError:
#                          print(f"JSON Decode Error (ContentTypeError) for DOI {doi}. Status: {response.status}. Returning None.")
#                          return doi, None
#                     except Exception as json_err: # Catch other potential JSON errors
#                          print(f"JSON Parsing Error for DOI {doi}: {json_err}. Returning None.")
#                          return doi, None
#                 elif response.status == 404:
#                     # print(f"DOI not found (404): {doi}")
#                     return doi, None # Not found, don't retry
#                 elif response.status == 429:
#                     print(f"Rate limit hit (429) for {doi}. Retrying in {current_delay}s (Attempt {attempt+1}/{max_retries})...")
#                     retry_after = response.headers.get("Retry-After")
#                     sleep_time = int(retry_after) if retry_after else current_delay
#                     await asyncio.sleep(sleep_time)
#                     current_delay *= 2 # Exponential backoff
#                 else:
#                     # print(f"HTTP Error {response.status} for DOI {doi}")
#                     response.raise_for_status() # Raise error for other bad statuses
#                     return doi, None # Should not be reached if raise_for_status works

#         except (aiohttp.ClientError, asyncio.TimeoutError) as e:
#             print(f"Network/Timeout Error for DOI {doi}: {e}. Retrying in {current_delay}s (Attempt {attempt+1}/{max_retries})...")
#             await asyncio.sleep(current_delay)
#             current_delay *= 2
#         except Exception as e:
#             print(f"Unexpected error processing DOI {doi}: {e}")
#             return doi, None # Don't retry on unexpected errors

#     print(f"Failed to fetch DOI {doi} after {max_retries} attempts.")
#     return doi, None


# async def process_chunk_async(doi_chunk_series):
#     """
#     Processes a chunk of DOIs asynchronously.
#     """
#     connector = aiohttp.TCPConnector(limit=CONCURRENCY_LIMIT)
#     async with aiohttp.ClientSession(connector=connector) as session:
#         tasks = [asyncio.create_task(fetch_journal_async(session, doi)) for doi in doi_chunk_series]
#         print(f"  Created {len(tasks)} tasks for this chunk. Fetching...")
#         # Gather results, which will be tuples of (doi, journal_name)
#         results = await asyncio.gather(*tasks)
#         print(f"  Chunk fetching complete.")
#         return results # List of (doi, journal_name_or_None) tuples

# # --- Main Processing Logic with Chunking and Saving ---

# # Split DataFrame into chunks
# list_of_df_chunks = np.array_split(df, len(df) // CHUNK_SIZE + 1)
# print(f"\nSplit DataFrame into {len(list_of_df_chunks)} chunks of approx size {CHUNK_SIZE}.")

# all_results_map = {} # Use a dictionary to store results {doi: journal}

# # --- Check if output file exists to potentially resume ---
# if os.path.exists(OUTPUT_FILE):
#     print(f"--- Found existing results file: {OUTPUT_FILE}. Loading previous results. ---")
#     try:
#         df_existing = pd.read_csv(OUTPUT_FILE)
#         # Handle potential missing 'doi' column or empty file
#         if 'doi' in df_existing.columns and not df_existing.empty:
#              all_results_map = pd.Series(df_existing.journal.values, index=df_existing.doi).to_dict()
#              print(f"--- Loaded {len(all_results_map)} results from file. ---")
#         else:
#              print(f"--- Existing file is empty or missing 'doi' column. Starting fresh. ---")
#     except pd.errors.EmptyDataError:
#         print(f"--- Existing file {OUTPUT_FILE} is empty. Starting fresh. ---")
#     except Exception as e:
#         print(f"--- Error loading existing file {OUTPUT_FILE}: {e}. Starting fresh. ---")
# # -------------------------------------------------------


# total_start_time = time.time()

# for i, chunk_df in enumerate(list_of_df_chunks):
#     chunk_start_time = time.time()
#     print(f"\n--- Processing Chunk {i+1}/{len(list_of_df_chunks)} ---")

#     # --- Filter out DOIs already processed ---
#     dois_in_chunk = chunk_df['doi'].dropna().unique() # Get unique non-null DOIs
#     dois_to_process = [doi for doi in dois_in_chunk if doi not in all_results_map]
#     # -----------------------------------------

#     if not dois_to_process:
#         print(f"--- All DOIs in this chunk already processed. Skipping. ---")
#         continue # Skip to the next chunk

#     print(f"--- Need to process {len(dois_to_process)} DOIs in this chunk. ---")

#     # --- Run async processing for the filtered DOIs ---
#     # Use await directly if in Jupyter/IPython, otherwise use asyncio.run()
#     try:
#         # Pass the list of DOIs to process
#         chunk_results_tuples = await process_chunk_async(dois_to_process)
#         # Or: chunk_results_tuples = asyncio.run(process_chunk_async(dois_to_process))

#         # Update the main results map
#         for doi, journal in chunk_results_tuples:
#             if doi: # Ensure DOI is not None before adding
#                  all_results_map[doi] = journal

#     except Exception as e:
#         print(f"!!!!!!!!!! ERROR processing chunk {i+1}: {e} !!!!!!!!!!!")
#         print("Saving progress before potentially stopping...")
#         # Fall through to save progress even if an error occurred in gather/run

#     # --- Save Progress After Each Chunk ---
#     try:
#         # Convert results map back to DataFrame for saving
#         df_to_save = pd.DataFrame(list(all_results_map.items()), columns=['doi', 'journal'])
#         df_to_save.to_csv(OUTPUT_FILE, index=False)
#         print(f"--- Progress saved to {OUTPUT_FILE} ({len(all_results_map)} total results). ---")
#     except Exception as e:
#         print(f"--- ERROR saving progress to {OUTPUT_FILE}: {e} ---")
#     # ------------------------------------

#     chunk_end_time = time.time()
#     print(f"--- Chunk {i+1} processing took: {chunk_end_time - chunk_start_time:.2f} seconds ---")

#     # --- Pause Between Chunks ---
#     if i < len(list_of_df_chunks) - 1: # Don't sleep after the last chunk
#         print(f"--- Pausing for {DELAY_BETWEEN_CHUNKS} seconds before next chunk... ---")
#         time.sleep(DELAY_BETWEEN_CHUNKS)
#     # --------------------------

# total_end_time = time.time()
# print(f"\n\n======================================================")
# print(f"Total processing finished in: {(total_end_time - total_start_time) / 60:.2f} minutes")
# print(f"Final results saved to {OUTPUT_FILE}")
# print(f"Total unique DOIs processed and saved: {len(all_results_map)}")
# print(f"======================================================")

# # --- Finally, map results back to the original DataFrame (optional) ---
# # This can be done after the script finishes by loading the CSV
# print("\nMapping results back to the original DataFrame...")
# df_final_results = pd.read_csv(OUTPUT_FILE)
# # Create a mapping series
# journal_map = pd.Series(df_final_results.journal.values, index=df_final_results.doi)
# # Map values, keeping existing NaNs if DOI wasn't found or processed
# df['journal'] = df['doi'].map(journal_map)

# print("Mapping complete.")
# print(df.head())
# print(f"\nJournal names found in final DataFrame: {df['journal'].notna().sum()}")
# print(f"Nulls (Original NaN, Not Found, Errors): {df['journal'].isna().sum()}")

In [ ]:
# Deprecated after finalization of "doi_journal_results.csv"
# # --- Reprocess Nulls ---
# print("\n--- Reprocessing Nulls ---")

# # Filter for DOIs with null journal names
# null_dois = df[df['journal'].isna()]['doi'].dropna().unique()

# if len(null_dois) == 0:
#     print("No null DOIs found. All records are complete.")
# else:
#     print(f"Found {len(null_dois)} DOIs with null journal names. Reprocessing...")

#     # Split null DOIs into chunks
#     null_chunks = np.array_split(null_dois, len(null_dois) // CHUNK_SIZE + 1)
#     print(f"Split null DOIs into {len(null_chunks)} chunks of approx size {CHUNK_SIZE}.")

#     for i, null_chunk in enumerate(null_chunks):
#         print(f"\n--- Processing Null Chunk {i+1}/{len(null_chunks)} ---")
#         try:
#             # Process the chunk asynchronously
#             null_results = await process_chunk_async(null_chunk)

#             # Update the results map with new data
#             for doi, journal in null_results:
#                 if doi:  # Ensure DOI is not None before adding
#                     all_results_map[doi] = journal

#             # Save updated results to the CSV file
#             df_to_save = pd.DataFrame(list(all_results_map.items()), columns=['doi', 'journal'])
#             df_to_save.to_csv(OUTPUT_FILE, index=False)
#             print(f"--- Progress saved to {OUTPUT_FILE} ({len(all_results_map)} total results). ---")

#         except Exception as e:
#             print(f"!!!!!!!!!! ERROR processing null chunk {i+1}: {e} !!!!!!!!!!!")
#             print("Saving progress before potentially stopping...")
#             # Save progress even if an error occurs
#             df_to_save = pd.DataFrame(list(all_results_map.items()), columns=['doi', 'journal'])
#             df_to_save.to_csv(OUTPUT_FILE, index=False)
#             print(f"--- Progress saved to {OUTPUT_FILE} ({len(all_results_map)} total results). ---")

#     print("\n--- Reprocessing Nulls Complete ---")

# # Map the updated results back to the DataFrame
# df_final_results = pd.read_csv(OUTPUT_FILE)
# journal_map = pd.Series(df_final_results.journal.values, index=df_final_results.doi)
# df['journal'] = df['doi'].map(journal_map)

# print("\nMapping complete.")
# print(df.head())
# print(f"\nJournal names found in final DataFrame: {df['journal'].notna().sum()}")
# print(f"Nulls (Original NaN, Not Found, Errors): {df['journal'].isna().sum()}")

#### Cleaning

In [ ]:
# # Removing entries where doi could not be associated with a journal and journal is NaN
# print(f'DataFrame dimension before removing NaN values in Journals: {df.shape}')

# df = df[~df["journal"].isna()]

# print(f'DataFrame dimension after removing NaN values in Journals: {df.shape}')

### Type - DEPRECATED -

Retriving types from Crossref API by doi.

In [ ]:
# # --- Configuration Parameters ---
# CHUNK_SIZE = 100          # How many DOIs to process in each batch
# CONCURRENCY_LIMIT = 4     # Max simultaneous requests
# DELAY_PER_REQUEST = 0.01   # Seconds to wait BEFORE each request attempt (e.g., 0.3 = ~3 req/sec max)
# RETRY_INITIAL_DELAY = 5   # Seconds to wait after first 429 error
# MAX_RETRIES = 3           # Max attempts for a single DOI
# DELAY_BETWEEN_CHUNKS = 0.5 # Seconds to pause between processing chunks
# OUTPUT_FILE = 'doi_type_results.csv' # File to save progress
# USER_AGENT = 'mailto:martina.esser@iu-study.org; purpose: Academic Research Mapping DOIs, Feature Engineering for a Study Project'
# # -----------------------------

# async def fetch_journal_async(session, doi, max_retries=MAX_RETRIES, initial_delay=RETRY_INITIAL_DELAY):
#     """
#     Asynchronously fetches the type for a given DOI using the Crossref API.
#     Includes retry logic, pre-request delay, and respects configured parameters.
#     Returns a tuple (doi, item_type). item_type is None if not found or on error.
#     """
#     # --- Polite Pre-Request Delay ---
#     await asyncio.sleep(DELAY_PER_REQUEST)
#     # -------------------------------

#     if pd.isna(doi):
#         # Return None for type if DOI is NaN
#         return doi, None

#     url = f"https://api.crossref.org/works/{doi}"
#     headers = {'User-Agent': USER_AGENT}

#     current_delay = initial_delay
#     for attempt in range(max_retries):
#         try:
#             async with session.get(url, headers=headers, timeout=30) as response:
#                 if response.status == 200:
#                     try:
#                         data = await response.json()
#                         # --- CORRECTED LINE ---
#                         # Get the 'type' string directly, default to None if not found
#                         item_type = data.get('message', {}).get('type', None)
#                         # --- Return the full type string ---
#                         return doi, item_type
#                     except aiohttp.ContentTypeError:
#                          print(f"JSON Decode Error (ContentTypeError) for DOI {doi}. Status: {response.status}. Returning None.")
#                          return doi, None
#                     except Exception as json_err:
#                          print(f"JSON Parsing Error for DOI {doi}: {json_err}. Returning None.")
#                          return doi, None
#                 elif response.status == 404:
#                     # DOI not found, return None for type
#                     # print(f"DOI {doi} not found (404).") # Optional: uncomment for verbose logging
#                     return doi, None
#                 elif response.status == 429:
#                     print(f"Rate limit hit (429) for {doi}. Retrying in {current_delay}s (Attempt {attempt+1}/{max_retries})...")
#                     retry_after = response.headers.get("Retry-After")
#                     # Use Retry-After header if available, otherwise use exponential backoff
#                     sleep_time = int(retry_after) if retry_after and retry_after.isdigit() else current_delay
#                     await asyncio.sleep(sleep_time)
#                     current_delay *= 2 # Exponential backoff for next potential 429
#                 else:
#                     # Log other HTTP errors but treat as 'not found' for this purpose
#                     print(f"HTTP Error {response.status} for DOI {doi}. Treating as not found.")
#                     # response.raise_for_status() # Optionally raise error for other bad statuses
#                     return doi, None # Return None for type on other HTTP errors

#         except (aiohttp.ClientError, asyncio.TimeoutError) as e:
#             print(f"Network/Timeout Error for DOI {doi}: {e}. Retrying in {current_delay}s (Attempt {attempt+1}/{max_retries})...")
#             await asyncio.sleep(current_delay)
#             current_delay *= 2 # Exponential backoff for network issues
#         except Exception as e:
#             print(f"Unexpected error processing DOI {doi}: {e}")
#             # Don't retry on unexpected errors, return None for type
#             return doi, None

#     print(f"Failed to fetch DOI {doi} after {max_retries} attempts.")
#     # Failed after all retries, return None for type
#     return doi, None


# async def process_chunk_async(doi_chunk_list):
#     """
#     Processes a chunk (list) of DOIs asynchronously.
#     """
#     # Limit concurrency at the connector level
#     connector = aiohttp.TCPConnector(limit=CONCURRENCY_LIMIT)
#     async with aiohttp.ClientSession(connector=connector) as session:
#         tasks = [asyncio.create_task(fetch_journal_async(session, doi)) for doi in doi_chunk_list]
#         print(f"  Created {len(tasks)} tasks for this chunk. Fetching...")
#         # Gather results, which will be tuples of (doi, item_type_or_None)
#         results = await asyncio.gather(*tasks)
#         print(f"  Chunk fetching complete.")
#         return results # List of (doi, item_type_or_None) tuples

# # --- Main Processing Logic with Chunking and Saving ---

# # Split DataFrame into chunks based on the 'doi' column
# # Ensure we handle potential NaNs gracefully during splitting if needed, though processing logic handles them
# list_of_doi_chunks = np.array_split(df['doi'].dropna().unique(), len(df['doi'].dropna().unique()) // CHUNK_SIZE + 1)
# print(f"\nSplit unique non-null DOIs into {len(list_of_doi_chunks)} chunks of max size {CHUNK_SIZE}.")

# all_results_map = {} # Dictionary to store {doi: item_type}

# # --- Check if output file exists to potentially resume ---
# if os.path.exists(OUTPUT_FILE):
#     print(f"--- Found existing results file: {OUTPUT_FILE}. Loading previous results. ---")
#     try:
#         df_existing = pd.read_csv(OUTPUT_FILE)
#         # Handle potential missing 'doi'/'type' columns or empty file
#         if 'doi' in df_existing.columns and 'type' in df_existing.columns and not df_existing.empty:
#              # Convert loaded data to dictionary, handling potential NaNs in the 'type' column
#              all_results_map = pd.Series(df_existing.type.values, index=df_existing.doi).where(pd.notna, None).to_dict()
#              print(f"--- Loaded {len(all_results_map)} results from file. ---")
#         else:
#              print(f"--- Existing file is empty or missing 'doi'/'type' columns. Starting fresh. ---")
#              all_results_map = {} # Ensure it's empty if file is invalid
#     except pd.errors.EmptyDataError:
#         print(f"--- Existing file {OUTPUT_FILE} is empty. Starting fresh. ---")
#         all_results_map = {}
#     except Exception as e:
#         print(f"--- Error loading existing file {OUTPUT_FILE}: {e}. Starting fresh. ---")
#         all_results_map = {}
# # -------------------------------------------------------


# total_start_time = time.time()

# for i, doi_chunk in enumerate(list_of_doi_chunks):
#     chunk_start_time = time.time()
#     print(f"\n--- Processing Chunk {i+1}/{len(list_of_doi_chunks)} ---")

#     # --- Filter out DOIs already processed ---
#     dois_to_process = [doi for doi in doi_chunk if pd.notna(doi) and doi not in all_results_map]
#     # -----------------------------------------

#     if not dois_to_process:
#         print(f"--- All DOIs in this chunk already processed or invalid. Skipping. ---")
#         continue # Skip to the next chunk

#     print(f"--- Need to process {len(dois_to_process)} DOIs in this chunk. ---")

#     try:
#         chunk_results_tuples = await process_chunk_async(dois_to_process) # List of (doi, item_type)

#         # Update the main results map
#         for doi, item_type in chunk_results_tuples:
#             if pd.notna(doi): # Ensure DOI is not None/NaN before adding
#                  all_results_map[doi] = item_type # item_type can be None if fetch failed/404

#     except RuntimeError as e:
#          if "cannot run current event loop" in str(e) and 'await' in open(__file__).read():
#               print("\nERROR: Detected use of 'await' outside an async function or compatible environment (like Jupyter).")
#               print("If running as a standard .py script, please modify the script to use 'asyncio.run(process_chunk_async(dois_to_process))' instead of 'await process_chunk_async(...)' in the main loop.")
#               print("Stopping execution.")
#               exit()
#          else:
#               print(f"!!!!!!!!!! RUNTIME ERROR processing chunk {i+1}: {e} !!!!!!!!!!!")
#               print("Saving progress before potentially stopping...")
#     except Exception as e:
#         print(f"!!!!!!!!!! UNEXPECTED ERROR processing chunk {i+1}: {e} !!!!!!!!!!!")
#         print("Saving progress before potentially stopping...")

#     # --- Save Progress After Each Chunk ---
#     try:
#         df_to_save = pd.DataFrame(list(all_results_map.items()), columns=['doi', 'type'])
#         df_to_save.to_csv(OUTPUT_FILE, index=False)
#         print(f"--- Progress saved to {OUTPUT_FILE} ({len(all_results_map)} total results). ---")
#     except Exception as e:
#         print(f"--- ERROR saving progress to {OUTPUT_FILE}: {e} ---")
#     # ------------------------------------

#     chunk_end_time = time.time()
#     print(f"--- Chunk {i+1} processing took: {chunk_end_time - chunk_start_time:.2f} seconds ---")

#     # --- Pause Between Chunks ---
#     if i < len(list_of_doi_chunks) - 1: # Don't sleep after the last chunk
#         print(f"--- Pausing for {DELAY_BETWEEN_CHUNKS} seconds before next chunk... ---")
#         time.sleep(DELAY_BETWEEN_CHUNKS)
#     # --------------------------

# total_end_time = time.time()
# print(f"\n\n======================================================")
# print(f"Total processing finished in: {(total_end_time - total_start_time) / 60:.2f} minutes")
# print(f"Final results saved to {OUTPUT_FILE}")
# print(f"Total unique DOIs processed and saved: {len(all_results_map)}")
# print(f"======================================================")

# print("\nMapping results back to the original DataFrame...")
# try:
#     df_final_results = pd.read_csv(OUTPUT_FILE)
#     type_map = pd.Series(df_final_results.type.values, index=df_final_results.doi)

#     # Map values to a new 'type' column in the original df
#     df['type'] = df['doi'].map(type_map)

#     print("Mapping complete.")
#     print("\nDataFrame head with mapped types:")
#     print(df.head())
#     print(f"\nPublication types found in final DataFrame: {df['type'].notna().sum()}")
#     print(f"Nulls (Original NaN DOI, Not Found, Errors, or DOI not processed): {df['type'].isna().sum()}")
#     print(f"\nValue counts for fetched types:")
#     print(df['type'].value_counts(dropna=False)) # Show counts including NaNs

# except FileNotFoundError:
#     print(f"Error: Output file {OUTPUT_FILE} not found. Cannot map results back.")
# except Exception as e:
#     print(f"Error mapping results back to DataFrame: {e}")



#### Cleaning

Not necessary, no NaNs in types after removal of journal NaNs

### Flatten Domain and Area

In [ ]:
# Split the 'Domain' column into individual domains and explode into rows
df_domains = df['Domain'].str.split(';').explode().str.strip()

# Get all unique domains
unique_domains = df_domains.unique()

# Create binary columns for each unique domain
for domain in unique_domains:
    df[domain] = df['Domain'].str.contains(domain, case=False, na=False)

In [ ]:
# Split the 'Area' column into individual areas and explode into rows
df_areas = df['Area'].str.split(';').explode().str.strip()

# Get all unique areas
unique_areas = df_areas.unique()

# Create a DataFrame with binary columns for each unique area (more memory efficient, many columns...)
binary_columns = {area: df['Area'].str.contains(area, case=False, na=False) for area in unique_areas}
binary_df = pd.DataFrame(binary_columns)

# Concatenate the original DataFrame with the new binary columns
df = pd.concat([df, binary_df], axis=1)

## Text derived Features

### Abstract

#### Pre-Processing

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    return text

df['abstract'] = df['abstract'].fillna('')

df['abstract'] = df['abstract'].apply(preprocess_text)

Lemmatization

In [ ]:
def lemmatize_text(text: str) -> str:
    if pd.isna(text) or not isinstance(text, str):
        return ""  # Return empty string for NaN or non-string inputs
    
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text.lower())  # Lowercase and tokenize
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

df['abstract'] = df['abstract'].apply(lemmatize_text)


Standardize Abbreviations

In [ ]:
print("\n--- Standardizing Abbreviations ---")
abbreviation_map = {
    # CS / General ML
    'machine learning ml': 'machine learning',
    'learning ml': 'machine learning',
    'ml models': 'machine learning model',
    'reinforcement learning rl': 'reinforcement learning',
    'artificial intelligence ai': 'artificial intelligence',
    'internet things iot': 'internet things',
    'convolutional neural network cnn': 'convolutional neural network',
    'deep learning dl': 'deep learning',
    'deep neural network dnn': 'deep neural network',
    'neural networks cnns': 'neural networks',
    'language models llms': 'language model',
    'language model lm': 'language model',
    'language processing nlp': 'natural language processing',
    'natural language processing nlp': 'natural language processing',
    'federated learning fl': 'federated learning',
    'principal component analysis pca': 'principal component analysis',
    'structural equation modelling sem': 'structural equation modelling',
    'extended reality xr': 'extended reality',
    # Physics
    'black hole bh': 'black hole',
    'dark matter dm': 'dark matter',
    'density functional theory dft': 'density functional theory',
    'molecular dynamics md': 'molecular dynamics',
    # EESS
    'computed tomography ct': 'computed tomography',
    'magnetic resonance imaging mri': 'magnetic resonance imaging',
    'electroencephalography eeg': 'electroencephalography',
    'base station bs': 'base station',
    'channel state information csi': 'channel state information',
    'multipleinput multipleoutput mimo': 'multipleinput multipleoutput',
    'signaltonoise ratio snr': 'signaltonoise ratio',
    'automatic speech recognition asr': 'automatic speech recognition',
    'massive machinetype communications mmtc': 'massive machinetype communications',
    'reconfigurable intelligent surface ris': 'reconfigurable intelligent surface',
    'user equipment ue': 'user equipment',
    # QB
    # molecular dynamics md (already listed)
    # magnetic resonance imaging mri (already listed)
    # electroencephalography eeg (already listed)
    # artificial intelligence ai (already listed)
    # Econ
    # structural equation modelling sem (already listed)
    # QF
    'agentbased models abms': 'agentbased models',
    'agentbased model gabm': 'agentbased model',
    'environmental social governance esg': 'environmental social governance',
    'value risk var': 'value risk',
}

def standardize_abbreviations(text, abbreviation_map):
    if pd.isna(text) or not isinstance(text, str):
        return text, 0  # Return as-is if NaN or not a string, with 0 replacements

    replacement_count = 0  # Track the number of replacements
    for abbr, full_form in abbreviation_map.items():
        if abbr in text:  # Check if the abbreviation exists in the text
            text = text.replace(abbr, full_form)  # Replace abbreviations with full forms
            replacement_count += 1  # Increment the counter for each replacement
    return text, replacement_count

# Apply the function to the "abstract" column and track replacements
adjustment_counts = []
df['abstract'], adjustment_counts = zip(*df['abstract'].apply(lambda x: standardize_abbreviations(x, abbreviation_map)))

# Print total adjustments
total_adjustments = sum(adjustment_counts)
print(f"Total number of adjustments made: {total_adjustments}")

In [ ]:
import pandas as pd
import numpy as np
import asyncio
import nest_asyncio
import aiohttp
import time
import os
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.utils import resample
import umap
import scipy.sparse
from typing import List, Dict, Tuple, Any, Set, Optional
import inflect
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.ticker as mtick
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import json
import pickle

In [ ]:
# Checkpoint
#df.to_parquet("checkpoint_lemmatized.parquet", engine="pyarrow", index=False)

df = pd.read_parquet("checkpoint_lemmatized.parquet", engine="pyarrow")

#### Keyword Extraction (per domain and month)

In [ ]:
def extract_monthly_top_keywords(
    dataframe: pd.DataFrame,
    category_column: str,
    date_column: str = 'first_date',
    text_column: str = 'abstract',
    top_n: int = 30,
    boost_bigrams: float = 1.0,
    boost_trigrams: float = 2.0,
    max_df: float = 0.8,
    min_df: int = 1,
    ngram_range: Tuple[int, int] = (2, 3)
) -> Tuple[List[str], Dict[Any, List[Tuple[str, float]]]]:
    """
    Analyzes text data within a specific category of a DataFrame, grouped by month,
    to extract the top N keywords using TF-IDF, with optional boosting for n-grams.

    Args:
        dataframe (pd.DataFrame): The input DataFrame containing the data.
        category_column (str): The name of the boolean column used to filter the
                               DataFrame for the relevant category.
        date_column (str): The name of the column containing datetime objects.
        text_column (str): The name of the column containing the text data (e.g., abstracts).
        top_n (int): The number of top keywords to extract for each month.
        boost_bigrams (float): Factor to boost the TF-IDF score of bigrams.
        boost_trigrams (float): Factor to boost the TF-IDF score of trigrams.
        tfidf_max_df (float): max_df parameter for TfidfVectorizer. Ignore terms
                              that appear in more than this fraction of documents.
        tfidf_min_df (int): min_df parameter for TfidfVectorizer. Ignore terms
                            that appear in less than this number of documents.
        tfidf_ngram_range (Tuple[int, int]): ngram_range parameter for TfidfVectorizer.

    Returns:
        Tuple[List[str], Dict[Any, List[Tuple[str, float]]]]:
            - A list of unique top keywords found across all months.
            - A dictionary where keys are the year-month periods and values are
              lists of (keyword, score) tuples for that month's top keywords.
    """

    # --- Input Validation ---
    if not isinstance(dataframe, pd.DataFrame):
        raise TypeError("Input 'dataframe' must be a pandas DataFrame.")
    if category_column not in dataframe.columns:
        raise ValueError(f"Column '{category_column}' not found in DataFrame.")
    if date_column not in dataframe.columns:
        raise ValueError(f"Column '{date_column}' not found in DataFrame.")
    if text_column not in dataframe.columns:
        raise ValueError(f"Column '{text_column}' not found in DataFrame.")
    if not pd.api.types.is_datetime64_any_dtype(dataframe[date_column]):
         try:
             # Attempt conversion if not already datetime
             dataframe[date_column] = pd.to_datetime(dataframe[date_column])
             print(f"Warning: Column '{date_column}' converted to datetime objects.")
         except Exception as e:
            raise TypeError(f"Column '{date_column}' must be of datetime type or convertible to it. Error: {e}")
    if not pd.api.types.is_bool_dtype(dataframe[category_column]):
        # Attempt conversion if possible (e.g., 0/1)
        try:
            dataframe[category_column] = dataframe[category_column].astype(bool)
            print(f"Warning: Column '{category_column}' converted to boolean type.")
        except Exception as e:
            raise TypeError(f"Column '{category_column}' must be of boolean type or convertible to it. Error: {e}")
    if not isinstance(top_n, int) or top_n <= 0:
        raise ValueError("'top_n' must be a positive integer.")

    # --- Data Preparation ---
    # Filter for the specified category and create a copy to avoid SettingWithCopyWarning
    df_filtered = dataframe[dataframe[category_column]].copy()

    # Handle potential NaNs in text column before processing
    df_filtered.dropna(subset=[text_column], inplace=True)
    df_filtered[text_column] = df_filtered[text_column].astype(str) # Ensure text is string

    # Create the year_month period
    df_filtered['year_month'] = df_filtered[date_column].dt.to_period('M')

    monthly_top_keywords = {} # Dictionary to store results {year_month: [(keyword, score), ...]}

    # Sort by year_month to process chronologically
    df_filtered = df_filtered.sort_values('year_month')

    # --- Group by Month and Analyze ---
    # Group by the created year_month period
    for year_month, group_df in df_filtered.groupby('year_month'):

        # Get the processed text data for the current month
        texts_this_month = group_df[text_column]

        # Skip if no valid (non-empty) texts for this month
        if texts_this_month.empty or texts_this_month.str.strip().eq('').all():
            print(f"No valid text data found for month {year_month}. Skipping.")
            monthly_top_keywords[year_month] = []
            continue

        try:
            # Initialize TF-IDF Vectorizer FOR THIS MONTH'S DATA
            vectorizer = TfidfVectorizer(
                stop_words='english',
                max_df=max_df,
                min_df=min_df,
                ngram_range=ngram_range
            )

            # Fit and transform the texts *for this month*
            tfidf_matrix = vectorizer.fit_transform(texts_this_month)

            # Get the feature names (keywords) learned from this month's data
            feature_names = vectorizer.get_feature_names_out()

            # Calculate the sum of TF-IDF scores for each term across all docs in this month
            sum_tfidf = tfidf_matrix.sum(axis=0)

            # Map scores to feature names
            scores = [(feature_names[col], sum_tfidf[0, col]) for col in range(tfidf_matrix.shape[1])]

            # Boost N-grams
            boosted_scores = []
            for term, score in scores:
                num_spaces = term.count(' ')
                boosted_score = score
                if num_spaces == 1: 
                    boosted_score *= boost_bigrams
                elif num_spaces == 2: 
                    boosted_score *= boost_trigrams

                if boosted_score > 0: # Only add terms with a positive boosted score
                     boosted_scores.append((term, boosted_score))

            # Sort terms by the potentially boosted score in descending order
            sorted_scores = sorted(boosted_scores, key=lambda x: x[1], reverse=True)

            # Get the top N keywords for the month
            top_keywords_this_month = sorted_scores[:top_n]

            # Store the results
            monthly_top_keywords[year_month] = top_keywords_this_month

        except ValueError as e:
            # Handle cases where TF-IDF might fail (e.g., all terms are stop words after filtering)
            print(f"Could not process month {year_month} with TF-IDF: {e}")
            monthly_top_keywords[year_month] = []

    # --- Collect Unique Keywords ---
    unique_keywords_set = set()
    for keywords in monthly_top_keywords.values():
        unique_keywords_set.update([keyword for keyword, _ in keywords])

    unique_keywords_list = sorted(list(unique_keywords_set))

    return unique_keywords_list, monthly_top_keywords

In [ ]:
category_columns_to_process: List[str] = [
        "Physics",
        "Computer Science",
        "Statistics",
        "Mathematics",
        "Electrical Engineering and Systems Science",
        "Quantitative Biology",
        "Economics",
        "Quantitative Finance"
]

domain_specific_params: Dict[str, Dict[str, Any]] = {
    "Physics": {"min_df": 0.0005, "max_df": 0.6, "top_n": 100}, # Many records in Physics domain
    "Computer Science": {"min_df": 0.0005, "max_df": 0.6, "top_n": 60}, # Many records in Computer Science domain
    "Statistics": {"min_df": 0.04, "max_df": 0.6},
    "Mathematics": {"min_df": 0.01, "max_df": 0.6},
    "Electrical Engineering and Systems Science": {"min_df": 0.04, "max_df": 0.6}, # Missing keywords mid 2015 - beginning of 2017 even with very unrestrictive parameters
    # https://info.arxiv.org/new/eess_announce.html  Electrical Engineering and Systems Science archive (eess) was introduced 18 September 2017
    "Quantitative Biology": {"min_df": 0.04, "max_df": 0.6},
    "Economics": {"min_df": 0.06, "max_df": 0.6},
    "Quantitative Finance": {"min_df": 0.06, "max_df": 0.6},
}

domain_keywords: Dict[str, List[str]] = {}

print("Starting keyword extraction across multiple categories...")

# Loop through each category
for category_col in category_columns_to_process:
    print(f"\nProcessing category: '{category_col}'...")

    # Check if the category column exists in the DataFrame
    if category_col not in df.columns:
        print(f"Warning: Column '{category_col}' not found in DataFrame. Skipping.")
        continue

    # Check if there's any data for this category
    if not df[category_col].any():
        print(f"No entries found for category '{category_col}'. Skipping.")
        continue

    try:
        # Get the domain-specific parameters for this category
        params = domain_specific_params.get(category_col, {})
        top_n = params.get("top_n", 30)
        min_df = params.get("min_df", 1)
        max_df = params.get("max_df", 0.8)
        ngram_range = params.get("ngram_range", (2, 3))

        # Call the keyword extraction function with the specific parameters
        unique_keywords_list, monthly_keywords_dict = extract_monthly_top_keywords(
            dataframe=df,
            category_column=category_col,
            date_column='first_date',
            text_column='abstract',
            top_n=top_n,
            min_df=min_df,
            max_df=max_df,
            ngram_range=ngram_range
        )

        # Store the unique keywords for this category in the dictionary
        domain_keywords[category_col] = unique_keywords_list

        print(f"Finished processing '{category_col}'. Added {len(unique_keywords_list)} unique keywords.")
        if not unique_keywords_list:
            print(f"(No keywords met the TF-IDF criteria for '{category_col}')")

    except (ValueError, TypeError) as e:
        print(f"Error processing category '{category_col}': {e}. Skipping this category.")
    except Exception as e:
        # Catch any other unexpected errors
        print(f"An unexpected error occurred while processing category '{category_col}': {e}. Skipping this category.")

print("\n--- Keyword Extraction Complete ---")

# Print the total number of keywords per domain
for domain, keywords in domain_keywords.items():
    print(f"Domain: {domain}, Total Keywords: {len(keywords)}")

In [ ]:
# Checkpoint a saving

with open('domain_keywords.json', 'w') as json_file:
    json.dump(domain_keywords, json_file, indent=4)
    

export_data = {
    "unique_domains": unique_domains.tolist(),
    "unique_areas": unique_areas.tolist(),
    "unique_keywords_list": unique_keywords_list
}

with open("unique_data.json", "w") as json_file:
    json.dump(export_data, json_file, indent=4)

In [ ]:
# Checkpoint b loading

import pandas as pd
import numpy as np
import asyncio
import nest_asyncio
import aiohttp
import time
import os
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.utils import resample
import umap
import scipy.sparse
from typing import List, Dict, Tuple, Any, Set, Optional
import inflect
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.ticker as mtick
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import json
import pickle

df = pd.read_parquet("checkpoint_lemmatized.parquet", engine="pyarrow")

with open('domain_keywords.json', 'r') as json_file:
    domain_keywords = json.load(json_file)

with open("unique_data.json", "r") as json_file:
    imported_data = json.load(json_file)

unique_domains = np.array(imported_data["unique_domains"])
unique_areas = np.array(imported_data["unique_areas"])
unique_keywords_list = imported_data["unique_keywords_list"]

#### Keyword Cleaning

In [ ]:
df_data = []


for domain, keyword_list in domain_keywords.items():
    if not isinstance(keyword_list, list):
        print(f"  - Warning: Keywords for domain '{domain}' is not a list. Skipping this domain for DataFrame.")
        continue
    if not keyword_list:
        print(f"  - No keywords for domain '{domain}'. This domain will have no entries in DataFrame.")
        # If you want to represent domains with no keywords, you could add a row with None or empty string for keyword
        # df_data.append({'domain': domain, 'keyword': None}) # Example
        continue
    for keyword in keyword_list:
        # Append a dictionary for each keyword, associating it with its domain
        df_data.append({'domain': domain, 'keyword': keyword})

domain_keywords_df = pd.DataFrame(df_data)

In [ ]:
# --- Remove Generic Phrases ---
print("\n--- Removing Generic Phrases ---")
generic_phrases_to_remove = [
    'et al', 'results indicate', 'results suggest', 'results demonstrate',
    'paper propose', 'paper present', 'work propose', 'introduce novel', 'propose new',
    'experimental results', 'numerical results', 'simulation results',
    'demonstrate effectiveness', 'effectiveness proposed', 'proposed method',
    'proposed approach', 'proposed model', 'proposed algorithm', 'proposed scheme',
    'recent years', 'widely used', 'commonly used', 'wide range', 'large number',
    'based on', 'compared to', 'address problem', 'paper deals', 'paper investigates',
    'paper study', 'study investigates', 'study shows', 'study aims', 'aim paper',
    'demonstrate proposed', 'results proposed', 'findings indicate', 'findings suggest',
    'existing methods', 'existing approaches', 'existing literature', 'previous work',
    'previous studies', 'recent work', 'recent studies', 'recent developments',
    'provide evidence', 'paper provide', 'study provides', 'present paper',
    'main result', 'key role', 'important role', 'crucial role', 'significant role',
    'better understanding', 'deeper understanding', 'valuable insights', 'shed light',
    'future research', 'possible future research', 'directions future research',
    'case study', 'numerical examples', 'experimental data', 'real data', 'real world',
    'publicly available', 'code available','good agreement', 'high accuracy', 'better performance',
    'superior performance', 'computational cost', 'computational complexity', 'computationally efficient',
    'state art', 'stateoftheart methods','stateoftheart performance','results obtained', 'results confirm',
    'results reveal', 'address challenges', 'address issue', 'address gap',
    'apply method', 'apply results', 'approach based', 'method based', 'framework based',
    'consider problem', 'develop novel', 'develop framework',
    'empirical evidence', 'empirical application', 'empirical analysis',
    'establish existence', 'evaluate performance', 'examine potential',
    'explain sustainability', 'explore key', 'extract important',
    'findings emphasize', 'focus specifically', 'gain insights',
    'highlight importance', 'illustrate method', 'implementable pricebased',
    'improve performance', 'increase probability', 'investigate effect',
    'make use', 'obtain new', 'outperforms existing',
    'play crucial role', 'plays important role', 'present comprehensive',
    'provide new', 'purpose paper', 'purpose research',
    'showcase potential', 'solve problem', 'study aimed', 'study demonstrates',
    'study examines', 'suggests evaluate', 'theoretical analysis', 'theoretical results',
    'understand relationship', 'use cases', 'using data', 'using numerical', 'et al phys',
    'data used', 'question answering', 'study propose', 'use case', 'present new',
    'evidence suggests', 'model used', 'plays crucial', 'plays crucial role', 'present novel',
    'propose novel', 'paves way', 'significant challenge', 'outperforms state-of-the-art', 'introduces novel',
    'prove existence', 'new results', 'previous results', 'new examples', 'new method',
    'novel approach', 'similar results', 'open problem', 'new proof', 'recent results',
    'closely related', 'previously known', 'recently introduced', 'new approach', 'present new',
    'gives rise', 'model based', 'extensive experiments', 'upper bound', 'recently proposed',
    'present study', 'standard model', 'work present', 'pave way', 'present results', 'present work',
    'provide valuable insights', 'results provide', 'results pave way', 'standard model sm', 'taken account', 
    'taking account', 'best practices', 'challenging task', 'demonstrate effectiveness proposed', 'demonstrate proposed method',
    'effectiveness proposed approach', 'effectiveness proposed model', 'effectiveness proposed method',
    'novel method', 'findings reveal', 'experimental results demonstrate', 'experimental results proposed', 'experiments conducted', 'experiments demonstrate',
    'extensive experimental results', 'extensive experiments conducted', 'extensive experiments demonstrate', 'extensive experiments realworld', 'paper consider',
    'paper explores', 'paper introduce novel', 'paper introduces novel', 'paper present novel', 'paper presents comprehensive',
    'paper presents new', 'paper presents novel', 'paper propose new', 'paper propose novel', 'paper proposes novel',
    'result suggest', 'result demonstrate', 'result indicate', 'result obtained', 'result pave way', 'result provide', 'result suggest',
    'address challenge', 'address challenge propose', 'address issue paper', 'address issue propose', 'address limitation propose',
    'existing approach', 'existing method', 'experimental research data', 'experimental result', 'experimental result demonstrate',
    'experimental result proposed', 'extensive experiment', 'paper address problem', 'paper describes', 'paper introduce',
    'paper introduce new', 'paper investigate', 'paper present new', 'paper study problem', 'present novel approach',
    'research performance', 'result demonstrate', 'result demonstrate proposed', 'result obtained', 'result proposed',
    'result proposed method', 'data set', 'data set different', 'data point', 'data based',
    'experimental result', 'data collection', 'introduce new', 'paper aim', 'paper analysis',
    'paper analyze', 'paper deal', 'paper introduces', 'paper investigate', 'paper model approximate',
    'paper provides', 'recent advance', 'recent development', 'recent year', 'recently used',
    'significantly outperforms', 'special case', 'open question', 'stateoftheart method', 'provide example',
    'provide explicit', 'provide insight', 'provide new proof', 'recent advancement', 'recent paper',
    'recent result', 'result hold', 'result numerical example', 'result obtained', 'result paper',
    'study conducted', 'study investigate', 'study investigated', 'study present', 'study reported'

    # Synonym / Incomplete expressions section
    'deep neural', 'large language', 'neural networks', 'data sets', 'machine learning algorithms',
    'models llms', 'convolutional neural', 'van der', 'der waals', 'deep neural', 'language models',
    'intelligence ai', 'deep learning models', 'learning models', 'resonance imaging', 'magnetic resonance',
    'network cnn', 'neural network cnn', 'learning dl', 'markov chain monte', 'chain monte carlo',
    'monte carlo mcmc', 'chain monte', 'carlo mcmc', 'natural language', 'quantum manybody',
    'manybody systems', 'galactic nuclei agn', 'galactic nuclei', 'active galactic', 'nuclei agn',
    'quantum information', 'information processing', 'carlo simulations', 'md simulations', 'blood glucose values',
    'computer vision tasks', 'datasets demonstrate', 'deep learningbased', 'deep reinforcement', 'learning model'
]

def remove_keywords_with_prefixes(df, column, prefixes):
    """
    Removes rows from the DataFrame where the specified column's values start with any of the given prefixes.

    Args:
        df (pd.DataFrame): The DataFrame to process.
        column (str): The column to check for prefixes.
        prefixes (list): A list of prefixes to check.

    Returns:
        pd.DataFrame: A new DataFrame with the filtered rows.
    """
    pattern = f"^({'|'.join(prefixes)})"
    mask = ~df[column].str.lower().str.match(pattern)
    return df[mask]


count_before_generic_removal = len(domain_keywords_df)
mask = ~domain_keywords_df['keyword'].str.lower().isin(generic_phrases_to_remove)
refined_df = domain_keywords_df[mask].copy()
print(f"Keyword count after removing generic phrases: {len(refined_df)} (Removed {count_before_generic_removal - len(refined_df)})")

count_before_prefix_removal = len(refined_df)
prefixes_to_remove = ['result', 'paper', 'provide', 'recent', 'address', 'model ', 'method ', 'significantly', 'consider ', 'considers ', 'considered ']
refined_df = remove_keywords_with_prefixes(refined_df, 'keyword', prefixes_to_remove)
print(f"Keyword count after removing prefixes: {len(refined_df)} (Removed {count_before_prefix_removal - len(refined_df)})")

# --- Display final counts ---
print("\nFinal keyword counts per domain:")
print(refined_df['domain'].value_counts())

In [ ]:
# Export refined_df to a text file
#refined_df.to_csv('refined_keywords.txt', sep='\t', index=False, header=True)

#### Keyword Visualization

##### Overall in categories

In [ ]:
custom_palette_hex = ["#89043d","#8a817c","#bcb8b1","#f4f3ee","#e0afa0"]

def palette_color_func(word, **kwargs):
    return random.choice(custom_palette_hex)

domain_keyword_frequencies: Dict[str, Dict[str, int]] = {}

# Loop through unique domains in the refined keyword DataFrame
for domain_name in refined_df['domain'].unique():
    print(f"\nProcessing: '{domain_name}'...")

    # Get refined keywords for this domain
    refined_keywords_list = refined_df[refined_df['domain'] == domain_name]['keyword'].tolist()

    # Filter original DataFrame for abstracts of this domain
    category_mask = df[domain_name] == True
    category_abstracts = df.loc[category_mask, "abstract"].astype(str).fillna('')

    if category_abstracts.empty or not refined_keywords_list:
        print(f"  - Skipping '{domain_name}' (no abstracts or keywords).")
        continue

    # Calculate frequencies
    frequencies = {}
    lower_abstracts_series = category_abstracts.str.lower()
    for keyword in refined_keywords_list:
        escaped_keyword = re.escape(keyword.lower())
        # Use regex=True because re.escape is used.
        count = lower_abstracts_series.str.count(escaped_keyword).sum()
        if count > 0:
            frequencies[keyword] = int(count)

    domain_keyword_frequencies[domain_name] = frequencies

    if not frequencies:
        print(f"  - Skipping '{domain_name}' (no keywords found in abstracts).")
        continue

    print(f"  - Generating word cloud for '{domain_name}'...")

    # Generate and display Word Cloud
    wordcloud_generator = WordCloud(
        width=1200,
        height=600,
        background_color='black',
        collocations=False,
        min_font_size=10,
        #colormap='magma',
        color_func=palette_color_func,
        random_state=None
    ).generate_from_frequencies(frequencies)

    plt.figure(figsize=(16, 8))
    plt.imshow(wordcloud_generator, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Domain: {domain_name}', fontsize=18)
    plt.tight_layout(pad=0)
    plt.show()

In [ ]:
# Export domain_keyword_frequencies to a JSON file
with open('domain_keyword_frequencies.json', 'w') as json_file:
    json.dump(domain_keyword_frequencies, json_file, indent=4)

# # Reimport domain_keyword_frequencies from the JSON file
# with open('domain_keyword_frequencies.json', 'r') as json_file:
#     domain_keyword_frequencies = json.load(json_file)

In [ ]:
# Remove unnecessary local variables
try:
    del category_abstracts, category_mask, count, count_before_generic_removal, count_before_prefix_removal
    del custom_palette_hex, df_data, domain, escaped_keyword
    del frequencies, generic_phrases_to_remove, imported_data, json_file
    del keyword, keywords, lower_abstracts_series, mask, prefixes_to_remove
    del refined_keywords_list, wordcloud_generator
except NameError as e:
    print(f"Variable not found: {e}")
else:
    print("All specified variables deleted successfully.")

##### Category specific - DEPRECATED - 

In [ ]:
# Filtering out keywords based on cross-domain frequency (keywords appearing in >1 domains more than 20 times)

freq_df = pd.DataFrame([{'domain': dom, 'keyword': kw, 'frequency': fq}
                        for dom, kw_fq_dict in domain_keyword_frequencies.items()
                        for kw, fq in kw_fq_dict.items()])

# 2. Identify keywords frequent (>=5) in >1 domain
domain_counts = freq_df.query('frequency >= 20').groupby('keyword')['domain'].nunique()
keywords_to_exclude = domain_counts[domain_counts > 1].index

# 3. Filter the original refined_df to exclude these keywords
domain_specific_keywords_df = refined_df[~refined_df['keyword'].isin(keywords_to_exclude)].reset_index(drop=True)

In [ ]:
for domain_name in domain_specific_keywords_df['domain'].unique():
    print(f"\nProcessing: '{domain_name}'...")

    # Get the list of domain-specific keywords for this domain
    specific_keywords_set = set(domain_specific_keywords_df[
        domain_specific_keywords_df['domain'] == domain_name
    ]['keyword'])

    # Get the original frequencies for ONLY these specific keywords
    # Check if the domain exists in the original frequency dictionary
    if domain_name not in domain_keyword_frequencies:
        print(f"  - Skipping '{domain_name}' (no original frequencies found).")
        continue

    # Filter the original frequencies for the current domain's specific keywords
    frequencies_for_cloud = {
        keyword: freq
        for keyword, freq in domain_keyword_frequencies[domain_name].items()
        if keyword in specific_keywords_set and freq > 0 
    }

    print(f"  - Generating word cloud for '{domain_name}' with {len(frequencies_for_cloud)} specific keywords...")

    # Generate and display Word Cloud using the filtered frequencies
    wordcloud_generator = WordCloud(
        width=1200,
        height=600,
        background_color='black',
        collocations=False,
        min_font_size=10,
        color_func=palette_color_func,
        random_state=None
    ).generate_from_frequencies(frequencies_for_cloud)

    # Display Plot
    plt.figure(figsize=(16, 8))
    plt.imshow(wordcloud_generator, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Domain: {domain_name}', fontsize=18)
    plt.tight_layout(pad=0)
    plt.show()

## Dimensionality Reduction and Clustering

### Default Variables and Initialisation

In [ ]:
TOP_N_KEYWORDS_PER_DOMAIN = 400 
UMAP_N_COMPONENTS = 15         
UMAP_N_NEIGHBORS = 10          
UMAP_MIN_DIST = 0.1            
UMAP_METRIC = 'euclidean'      
RANDOM_STATE = 42              
SILHOUETTE_SAMPLE_SIZE = 10000 
K_RANGE_FOR_EVALUATION = range(2, 25) 

### Features for Clustering

In [ ]:
# ----- Keyword Selection -----

domain_top_keywords_map: Dict[str, List[str]] = {}
# Set to store the global union of all top N keywords
final_keyword_list_set = set()

# Check if domain_keyword_frequencies exists and is populated
if 'domain_keyword_frequencies' not in locals() or not domain_keyword_frequencies:
    print("Error: 'domain_keyword_frequencies' dictionary not found or empty. Cannot select top keywords.")
    final_keyword_list = [] # Initialize empty list
else:
    # Iterate through the domains present in the frequency dictionary
    for domain_name, keyword_freqs in domain_keyword_frequencies.items():
        print(f"  Processing domain: {domain_name}")

        if not keyword_freqs:
            print(f"    No keywords/frequencies found for domain '{domain_name}'.")
            domain_top_keywords_map[domain_name] = [] # Store empty list for this domain
            continue

        # Sort keywords in this domain by frequency (descending)
        valid_items = [(kw, fq) for kw, fq in keyword_freqs.items() if isinstance(fq, (int, float))]
        sorted_keywords = sorted(valid_items, key=lambda item: item[1], reverse=True)

        # Select the top N keywords for this specific domain
        top_n_for_domain = [kw for kw, freq in sorted_keywords[:TOP_N_KEYWORDS_PER_DOMAIN]]
        print(f"    Selected {len(top_n_for_domain)} keywords for '{domain_name}'.")

        # Store this list in the map
        domain_top_keywords_map[domain_name] = top_n_for_domain

        # Add these keywords to the global set for vectorizer vocabulary
        final_keyword_list_set.update(top_n_for_domain)

    # Convert the final global set to a sorted list for consistent column order in TfidfVectorizer
    final_keyword_list = sorted(list(final_keyword_list_set))
    print(f"\nCreated map 'domain_top_keywords_map' with top keywords per domain.")
    print(f"Created global list 'final_keyword_list' with {len(final_keyword_list)} unique keywords across all domains (for vectorizer).")

In [ ]:
# ----- Create Binary Keyword Features -----

if not final_keyword_list:
     print("Warning: final_keyword_list is empty. No keyword features will be created.")
     keyword_cols_created = []
else:
    print(f"\n--- Creating Binary Features for {len(final_keyword_list)} Keywords ---")

    # Ensure abstract is string type and handle potential NaNs
    lower_abstracts = df['abstract'].astype(str).fillna('').str.lower()

    # --- Configure TfidfVectorizer ---
    max_ngram_length = 1
    if final_keyword_list:
         max_ngram_length = max(len(kw.split()) for kw in final_keyword_list)

    print(f"  Configuring TfidfVectorizer (max_ngram={max_ngram_length})...")
    vectorizer = TfidfVectorizer(
        vocabulary=final_keyword_list,
        lowercase=True,
        binary=True,
        use_idf=False,
        norm=None,
        ngram_range=(1, max_ngram_length)
    )

    print("  Applying vectorizer to abstracts...")
    X_keywords_sparse = vectorizer.fit_transform(lower_abstracts)
    print(f"  Vectorizer finished. Output shape (sparse): {X_keywords_sparse.shape}")

    # --- Create DataFrame from Sparse Matrix ---
    feature_names = vectorizer.get_feature_names_out()
    keyword_cols_created = [f'kw_{name}' for name in feature_names] # These are the columns added
    print(f"  Creating DataFrame from sparse matrix for {len(keyword_cols_created)} keyword features...")
    try:
        df_keywords = pd.DataFrame.sparse.from_spmatrix(
            X_keywords_sparse,
            index=df.index,
            columns=keyword_cols_created
        )
        print("  Keyword DataFrame created.")

        # --- Concatenate with original DataFrame ---
        cols_to_drop_from_df = [col for col in keyword_cols_created if col in df.columns]
        if cols_to_drop_from_df:
             print(f"  Dropping existing columns from df before concat: {cols_to_drop_from_df}")
             df = df.drop(columns=cols_to_drop_from_df)

        df = pd.concat([df, df_keywords], axis=1)
        # Convert sparse keyword columns to int (0/1) AFTER concat if needed by downstream steps
        # This might increase memory usage significantly.
        # df[keyword_cols_created] = df[keyword_cols_created].astype(int)
        print(f"Concatenated keyword features. New df shape: {df.shape}")

    except MemoryError:
        print("\nError: MemoryError encountered while creating keyword DataFrame.")
        print("Consider reducing TOP_N_KEYWORDS_PER_DOMAIN or using algorithms accepting sparse input.")
        keyword_cols_created = [] # Prevent errors later
    except Exception as e:
        print(f"\nAn unexpected error occurred creating keyword DataFrame: {e}")
        keyword_cols_created = []

In [ ]:
# ----- Final Feature Lists -----

# --- Metadata Features ---
metadata_features = ['number_of_authors'] + list(unique_areas)
# type_dummies = pd.get_dummies(df['type'], prefix='type', drop_first=False, dummy_na=False)
# df = pd.concat([df, type_dummies], axis=1)
# metadata_features.extend(type_dummies.columns.tolist())

# --- Keyword Features ---
print(f"Keyword features defined: {len(keyword_cols_created)} columns starting with 'kw_'")

In [ ]:
# Checkpoint a saving

for col_name in df.columns:
    if pd.api.types.is_sparse(df[col_name].dtype):
        df[col_name] = df[col_name].sparse.to_dense()

df.to_parquet("checkpoint_with_keywords.parquet", engine="pyarrow", index=False)

variables_to_export = {
    "domain_top_keywords_map": domain_top_keywords_map,
    "final_keyword_list": final_keyword_list,
    "final_keyword_list_set": list(final_keyword_list_set),
    "keyword_cols_created": keyword_cols_created,
    "metadata_features": metadata_features,
    "unique_areas": unique_areas.tolist(),
    "unique_domains": unique_domains.tolist(),
    "unique_keywords_list": unique_keywords_list,
}

with open("checkpoint_variables.pkl", "wb") as f:
    pickle.dump(variables_to_export, f)

# CONSIDER DELETING FROM HERE WHEN "Clustering.ipynb" IS FINALIZED

In [ ]:
# Checkpoint b loading

import pandas as pd
import numpy as np
import asyncio
import nest_asyncio
import aiohttp
import time
import os
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.utils import resample
from sklearn.decomposition import PCA
import umap
import scipy.sparse
from typing import List, Dict, Tuple, Any, Set, Optional
import inflect
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import json
import pickle
import hdbscan
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from pandas.tseries.offsets import DateOffset
import textwrap

df = pd.read_parquet("checkpoint_with_keywords.parquet", engine="pyarrow")
df = df.reset_index(drop=True)

keyword_column_names = [col for col in df.columns if col.startswith('kw_')]

for col_name in keyword_column_names:
    if col_name in df.columns:
        current_dtype = df[col_name].dtype
        if pd.api.types.is_numeric_dtype(current_dtype):
            df[col_name] = df[col_name].astype(pd.SparseDtype(current_dtype, fill_value=0.0))
        else:
            print(f"Column '{col_name}' is not numeric, skipping sparse conversion.")
    else:
        print(f"Keyword column '{col_name}' not found in loaded DataFrame.")


with open("checkpoint_variables.pkl", "rb") as f:
    loaded_variables = pickle.load(f)

domain_top_keywords_map = loaded_variables["domain_top_keywords_map"]
final_keyword_list = loaded_variables["final_keyword_list"]
final_keyword_list_set = set(loaded_variables["final_keyword_list_set"])
keyword_cols_created = loaded_variables["keyword_cols_created"]
metadata_features = loaded_variables["metadata_features"]
unique_areas = np.array(loaded_variables["unique_areas"])
unique_domains = np.array(loaded_variables["unique_domains"])
unique_keywords_list = loaded_variables["unique_keywords_list"]

In [ ]:
# Dictionary to store results per domain
domain_cluster_results = {}
# Add a new column for sub-cluster labels, initialize with NaN or -1
df['sub_cluster'] = pd.NA

### Overall Clustering

In [ ]:
import pandas as pd
import numpy as np
import pickle
import time

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import umap
import hdbscan

import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"

Configuration

In [ ]:
# --- File Paths ---
DATA_FILE = "checkpoint_with_keywords.parquet"
VARIABLES_FILE = "checkpoint_variables.pkl"

# --- Pipeline Parameters ---
# PCA Configuration
PCA_N_COMPONENTS = 150
RANDOM_STATE = 42

# UMAP Configuration (for clustering)
UMAP_CLUSTERING_PARAMS = {
    'n_neighbors': 30,
    'n_components': 15,
    'min_dist': 0.0,
    'metric': 'euclidean',
    'random_state': RANDOM_STATE,
    'verbose': True
}

# UMAP Configuration (for 2D visualization)
UMAP_VISUALIZATION_PARAMS = {
    'n_neighbors': 30,
    'n_components': 2,
    'min_dist': 0.1,
    'metric': 'euclidean',
    'random_state': RANDOM_STATE,
    'verbose': True
}

# HDBSCAN Configuration
HDBSCAN_PARAMS = {
    'min_cluster_size': 50,
    'min_samples': 25,
    'metric': 'euclidean',
    'cluster_selection_method': 'eom',
    'gen_min_span_tree': True
}

Data Loading and Feature Preparation

In [ ]:
print(f"Loading data from '{DATA_FILE}'...")
df = pd.read_parquet(DATA_FILE, engine="pyarrow")
print(f"DataFrame loaded. Shape: {df.shape}")

# Load the helper variables (column lists)
print(f"Loading feature lists from '{VARIABLES_FILE}'...")
with open(VARIABLES_FILE, "rb") as f:
    loaded_variables = pickle.load(f)
metadata_features = loaded_variables["metadata_features"]
unique_domains = np.array(loaded_variables["unique_domains"])
keyword_cols_created = loaded_variables["keyword_cols_created"]
print("Helper variables loaded.")

# Assemble the full feature set
print("Assembling the final feature set for clustering...")

# Ensure all feature columns exist in the DataFrame
domain_features = [col for col in unique_domains if col in df.columns]
meta_features = [col for col in metadata_features if col in df.columns]
keyword_features = [col for col in keyword_cols_created if col in df.columns]

# Combine all feature lists
all_features = meta_features + domain_features + keyword_features
all_features = sorted(list(set(all_features)))  # Get unique sorted list
print(f"Total number of features: {len(all_features)}")
print(f" - Metadata & Area features: {len(meta_features)}")
print(f" - Domain features: {len(domain_features)}")
print(f" - Keyword features: {len(keyword_features)}")

# Create the feature matrix X
X = df[all_features]

# Handle Missing Values
# Impute with median for numeric columns. This is a robust strategy.
if X.isnull().sum().sum() > 0:
    print("NaNs found. Imputing with column medians...")
    X = X.fillna(X.median())
else:
    print("No NaNs found in the feature matrix.")

# Convert to NumPy array for scikit-learn
print("Converting feature matrix to NumPy array...")
X_np = X.to_numpy(dtype=np.float32)
print(f"Final feature matrix 'X_np' created with shape: {X_np.shape}")

Dimensionality Reduction and Clustering Pipeline

In [ ]:
# --- Step 1: Scaling ---
print("--- Step 1: Scaling data with StandardScaler ---")
start_time = time.time()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_np)
end_time = time.time()
print(f"Scaling completed in {end_time - start_time:.2f} seconds.")

In [ ]:
# --- Step 2: PCA ---
print(f"\n--- Step 2: Applying PCA to reduce to {PCA_N_COMPONENTS} components ---")
start_time = time.time()
pca = PCA(n_components=PCA_N_COMPONENTS, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(X_scaled)
end_time = time.time()
print(f"PCA completed in {end_time - start_time:.2f} seconds.")
print(f"Shape after PCA: {X_pca.shape}")
print(f"Total explained variance by {pca.n_components_} components: {np.sum(pca.explained_variance_ratio_):.4f}")

In [ ]:
# --- Step 3: UMAP ---
print(f"\n--- Step 3: Applying UMAP to reduce to {UMAP_CLUSTERING_PARAMS['n_components']} dimensions for clustering ---")
start_time = time.time()
umap_reducer_clustering = umap.UMAP(**UMAP_CLUSTERING_PARAMS)
X_umap_clustering = umap_reducer_clustering.fit_transform(X_pca)
end_time = time.time()
print(f"UMAP for clustering completed in {end_time - start_time:.2f} seconds.")
print(f"Shape after UMAP: {X_umap_clustering.shape}")

In [ ]:
# --- Step 4: HDBSCAN Clustering ---
print("\n--- Step 4: Applying HDBSCAN to find clusters ---")
start_time = time.time()
clusterer = hdbscan.HDBSCAN(**HDBSCAN_PARAMS)
cluster_labels = clusterer.fit_predict(X_umap_clustering)
end_time = time.time()
print(f"HDBSCAN completed in {end_time - start_time:.2f} seconds.")

In [ ]:
# --- Add cluster labels to DataFrame ---
df['cluster_label'] = cluster_labels
n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
n_noise = np.sum(cluster_labels == -1)
print("\n--- Clustering Results ---")
print(f"Number of clusters found: {n_clusters}")
print(f"Number of noise points: {n_noise} ({n_noise / len(df) * 100:.2f}%)")

Visualization of Clusters

In [ ]:
# --- Create 2D UMAP embedding for visualization ---
print("\n--- Creating 2D UMAP embedding for visualization ---")
start_time = time.time()
umap_reducer_viz = umap.UMAP(**UMAP_VISUALIZATION_PARAMS)
X_umap_viz = umap_reducer_viz.fit_transform(X_pca)
end_time = time.time()
print(f"2D UMAP for visualization completed in {end_time - start_time:.2f} seconds.")

# Add 2D coordinates to DataFrame for plotting
df['umap_x'] = X_umap_viz[:, 0]
df['umap_y'] = X_umap_viz[:, 1]

# %%
print("--- Generating interactive cluster plot ---")

# Prepare data for Plotly
plot_df = df.copy()
plot_df['cluster_label_str'] = plot_df['cluster_label'].astype(str)
plot_df.loc[plot_df['cluster_label'] == -1, 'cluster_label_str'] = 'Noise'

# Get cluster sizes for the legend
cluster_sizes = plot_df['cluster_label_str'].value_counts().reset_index()
cluster_sizes.columns = ['cluster_label_str', 'count']
plot_df = pd.merge(plot_df, cluster_sizes, on='cluster_label_str')
plot_df['legend_entry'] = plot_df['cluster_label_str'] + ' (' + plot_df['count'].astype(str) + ')'

# Create the plot
fig = go.Figure()

In [ ]:
# Plot clustered points first
clustered_data = plot_df[plot_df['cluster_label'] != -1].sort_values('cluster_label')
fig.add_trace(go.Scattergl(
    x=clustered_data['umap_x'],
    y=clustered_data['umap_y'],
    mode='markers',
    marker=dict(
        color=clustered_data['cluster_label'],
        colorscale='Viridis',  # A nice colorscale for clusters
        showscale=False,
        size=3,
        opacity=0.7
    ),
    customdata=clustered_data[['id', 'title', 'cluster_label']],
    hovertemplate='<b>Title:</b> %{customdata[1]}<br>' +
                  '<b>ID:</b> %{customdata[0]}<br>' +
                  '<b>Cluster:</b> %{customdata[2]}<br>' +
                  'UMAP-X: %{x:.3f}<br>UMAP-Y: %{y:.3f}<extra></extra>',
    name='Clusters'
))

# Plot noise points on top, in grey
noise_data = plot_df[plot_df['cluster_label'] == -1]
fig.add_trace(go.Scattergl(
    x=noise_data['umap_x'],
    y=noise_data['umap_y'],
    mode='markers',
    marker=dict(
        color='lightgrey',
        size=2,
        opacity=0.4
    ),
    customdata=noise_data[['id', 'title']],
    hovertemplate='<b>Title:</b> %{customdata[1]}<br>' +
                  '<b>ID:</b> %{customdata[0]}<br>' +
                  '<b>Cluster:</b> Noise<br>' +
                  'UMAP-X: %{x:.3f}<br>UMAP-Y: %{y:.3f}<extra></extra>',
    name=f"Noise ({len(noise_data)})"
))

# Update layout
fig.update_layout(
    title=f'Global Clustering Results: {n_clusters} Clusters Found',
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2',
    height=800,
    legend_title_text='Cluster Labels',
    showlegend=True
)
fig.show()

# %% [markdown]
# ### Cluster Size Distribution
# A quick look at the distribution of records across the identified clusters.

# %%
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')

cluster_counts = df[df['cluster_label'] != -1]['cluster_label'].value_counts().sort_index()

plt.figure(figsize=(16, 8))
sns.barplot(x=cluster_counts.index, y=cluster_counts.values, palette='viridis')
plt.title('Number of Records per Cluster (excluding noise)', fontsize=16)
plt.xlabel('Cluster ID', fontsize=12)
plt.ylabel('Number of Records', fontsize=12)

if len(cluster_counts) > 50:
    plt.xticks(rotation=90, fontsize=8)
    # Show every 5th tick label to avoid clutter
    for index, label in enumerate(plt.gca().get_xticklabels()):
        if index % 5 != 0:
            label.set_visible(False)
else:
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


### Per Domain Clustering

#### Variables and Functions

In [ ]:
DEFAULT_UMAP_PARAMS = {
    'n_components': 15,
    'n_neighbors': 10,
    'min_dist': 0.1,
    'metric': 'euclidean',
    'random_state': 42,
    'verbose': True,
    #'low_memory': True
}
DEFAULT_KMEANS_PARAMS = {
    'init': 'k-means++',
    'n_init': 'auto',
    'max_iter': 300,
    'random_state': 42
}
DEFAULT_K_RANGE = range(2, 301)
DEFAULT_SILHOUETTE_SAMPLE_SIZE = 5000
RANDOM_STATE = 42
ROLLING_WINDOW = 6

EXCLUSION_COLOR = 'lightgrey'

COLORS = {
    0: '#e6194b',
    1: '#3cb44b',
    2: '#e6beff',
    3: '#4363d8',
    4: '#f58231',
    5: '#911eb4',
    6: '#46f0f0',
    7: '#f032e6',
    8: '#f6bb0c',
    9: '#fabebe',
    10: '#008080',
    11: '#808000',
    12: '#9a6324',
    13: '#aaffc3',
    14: '#ffd8b1',
    15: '#800000',
    16: '#ffe119',
    17: '#000075',
    18: '#a9a9ff',
    19: '#ff4500',
    20: '#000000',
    21: '#808080',
    22: '#bcf60c',
    23: '#fffac8',
    24: '#dda0dd',
    25: '#556b2f'
}

In [ ]:
def generate_random_colors(num_colors: int) -> dict:
    """
    Generates a dictionary of random hexadecimal color codes.

    Args:
        num_colors (int): The number of random colors to generate.

    Returns:
        dict: A dictionary where keys are integers (0 to num_colors-1) and values are hex color codes.
    """
    colors = {}
    for i in range(num_colors):
        # Generate a random color in hexadecimal format
        color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
        colors[i] = color
    return colors

COLORS_2 = generate_random_colors(300)

In [ ]:
# --- Functions to Prepare Data and Reduce Dimensionality ---
def prepare_domain_data(
    domain_name: str,
    df: pd.DataFrame,
    metadata_features: List[str],
    keyword_cols_created: List[str],
    domain_top_keywords_map: Dict[str, List[str]]
) -> Optional[Tuple[pd.DataFrame, np.ndarray, List[str]]]:
    """
    Filters data for a specific domain, selects features based on metadata
    and the Top N keywords specific to that domain, handles NaNs, and converts
    features to a dense NumPy array.

    Args:
        domain_name: The name of the domain column to filter by.
        df: The main DataFrame containing all data and features.
        metadata_features: List of metadata column names to include.
        keyword_cols_created: List of all 'kw_...' column names that were created.
        domain_top_keywords_map: Dictionary mapping domain name to its list of Top N keywords.

    Returns:
        A tuple containing:
        - df_domain: DataFrame subset for the domain.
        - X_domain_dense: Dense NumPy array of selected metadata and domain-specific keyword features.
        - final_features_used: List of the actual feature columns used (metadata + keywords).
        Returns None if no data or relevant features are found, or if errors occur.
    """
    print(f"\n--- Preparing Data for Domain: {domain_name} ---")

    # --- Filter Data ---
    if domain_name not in df.columns:
        print(f"  Error: Domain column '{domain_name}' not found in DataFrame.")
        return None
    domain_mask = df[domain_name] == True
    df_domain = df[domain_mask].copy()

    if df_domain.empty:
        print(f"  No data found for domain '{domain_name}'.")
        return None
    print(f"  Found {len(df_domain)} documents.")

    # --- Feature Selection ---

    # 1. Select Metadata Features
    # Ensure metadata columns exist in the df_domain subset
    selected_metadata_cols = [col for col in metadata_features if col in df_domain.columns]
    if len(selected_metadata_cols) != len(metadata_features):
        missing_meta = [col for col in metadata_features if col not in df_domain.columns]
        print(f"  Warning: The following metadata columns were requested but not found in df_domain: {missing_meta}")

    # 2. Select Domain-Specific Top N Keyword Features
    top_keywords_for_this_domain = domain_top_keywords_map.get(domain_name, [])

    if not top_keywords_for_this_domain and not selected_metadata_cols: # If no keywords AND no metadata
        print(f"  No Top N keywords defined for domain '{domain_name}' and no metadata features selected.")
        return None
    elif not top_keywords_for_this_domain:
        print(f"  Warning: No Top N keywords defined for domain '{domain_name}'. Proceeding with metadata only.")


    domain_specific_kw_cols = [
        f'kw_{kw}' for kw in top_keywords_for_this_domain
        if f'kw_{kw}' in keyword_cols_created
    ]

    missing_kw_cols = [
        f'kw_{kw}' for kw in top_keywords_for_this_domain
        if f'kw_{kw}' not in keyword_cols_created
    ]
    if missing_kw_cols:
        print(f"  Warning: The following Top N keyword columns were expected but not found: {missing_kw_cols}")

    if not domain_specific_kw_cols and not selected_metadata_cols:
        print(f"  No valid keyword columns found for Top N keywords and no metadata features for '{domain_name}'.")
        return None
    elif not domain_specific_kw_cols:
        print(f"  Warning: No valid keyword columns found for Top N keywords for '{domain_name}'. Proceeding with metadata only.")


    # 3. Combine Metadata and Keyword Features
    # Ensure no overlap between metadata and keyword column names (unlikely but good practice)
    final_features_used = selected_metadata_cols + [
        kw_col for kw_col in domain_specific_kw_cols if kw_col not in selected_metadata_cols
    ]

    print(f"  Using {len(selected_metadata_cols)} metadata features and "
          f"{len([kw_col for kw_col in domain_specific_kw_cols if kw_col not in selected_metadata_cols])} "
          f"domain-specific keyword features. Total: {len(final_features_used)} features.")

    # Ensure all selected features actually exist in df_domain before creating X_domain
    # This is a final check, especially if df_domain is a very small subset.
    valid_current_features = [col for col in final_features_used if col in df_domain.columns]
    if len(valid_current_features) != len(final_features_used):
         missing_from_subset = [col for col in final_features_used if col not in df_domain.columns]
         print(f"  Warning: Some selected features missing from df_domain subset (should be rare): {missing_from_subset}")
    if not valid_current_features:
         print(f"  Error: No valid feature columns left after checking df_domain.")
         return None

    X_domain = df_domain[valid_current_features]

    # --- Handle NaNs (More likely now with metadata) and Convert to Dense ---
    try:
        # Check for NaNs, especially in metadata columns
        has_nans = False
        # Iterate over a copy of columns to avoid issues if X_domain is modified
        for col in list(X_domain.columns):
            if pd.isna(X_domain[col]).any():
                has_nans = True
                print(f"  Warning: NaNs found in feature column '{col}'. Imputing...")
                if X_domain[col].dtype == object or pd.api.types.is_string_dtype(X_domain[col].dtype):
                    # For object/string columns (e.g., if a metadata feature was unexpectedly string)
                    # Impute with mode or a placeholder. For clustering, numeric is better.
                    # This case should ideally be handled by one-hot encoding earlier.
                    mode_val = X_domain[col].mode()
                    fill_value = mode_val[0] if not mode_val.empty else "Unknown"
                    print(f"    Imputing object column '{col}' with mode: {fill_value}")
                    X_domain[col] = X_domain[col].fillna(fill_value)
                    # If it's still object, it might cause issues with StandardScaler.
                    # Consider converting to categorical codes or one-hot encoding if this happens.
                elif pd.api.types.is_numeric_dtype(X_domain[col].dtype):
                    # For numeric columns (includes int, float, potentially sparse numeric)
                    median_val = X_domain[col].median()
                    print(f"    Imputing numeric column '{col}' with median: {median_val}")
                    X_domain[col] = X_domain[col].fillna(median_val)
                else:
                    # For other types (e.g. boolean if not 0/1 already)
                    # Impute with the most frequent value (mode) or 0/False
                    mode_val = X_domain[col].mode()
                    fill_value = mode_val[0] if not mode_val.empty else 0
                    print(f"    Imputing column '{col}' of type {X_domain[col].dtype} with mode: {fill_value}")
                    X_domain[col] = X_domain[col].fillna(fill_value)


        # Final check for NaNs after imputation
        if has_nans and pd.isna(X_domain).any().any():
             print("  Error: NaNs still present after imputation. Examine problematic columns.")
             # Identify columns with remaining NaNs
             remaining_nan_cols = X_domain.columns[X_domain.isna().any()].tolist()
             print(f"    Columns with remaining NaNs: {remaining_nan_cols}")
             return None


        print("  Converting feature matrix to dense NumPy array...")
        # StandardScaler expects float input, so convert.
        # Binary features (0/1) will remain 0.0/1.0.
        X_domain_dense = X_domain.to_numpy(dtype=np.float32)
        print(f"  Created dense feature matrix with shape: {X_domain_dense.shape}")

        # Return the list of columns actually used
        return df_domain, X_domain_dense, valid_current_features

    except MemoryError:
        print(f"  Error: MemoryError converting features for domain '{domain_name}' to dense array.")
        return None
    except Exception as e:
        print(f"  Error preparing features for domain '{domain_name}': {e}")
        return None

def reduce_dimensionality(
    X_domain_dense: np.ndarray,
    umap_params: Dict = DEFAULT_UMAP_PARAMS
) -> Optional[Tuple[np.ndarray, StandardScaler, umap.UMAP]]:
    """
    Applies StandardScaler and UMAP to the dense feature matrix.

    Args:
        X_domain_dense: Dense NumPy array of features for the domain.
        umap_params: Dictionary of parameters for UMAP.

    Returns:
        A tuple containing:
        - X_domain_reduced_umap: NumPy array of reduced features.
        - scaler_domain: The fitted StandardScaler object.
        - reducer_domain: The fitted UMAP object.
        Returns None if input is invalid.
    """
    print("\n--- Scaling and Applying UMAP ---")
    if X_domain_dense is None or X_domain_dense.shape[0] == 0:
        print("  Error: Input feature matrix is empty or None.")
        return None

    try:
        # --- Scaling ---
        print("  Scaling domain features...")
        scaler_domain = StandardScaler()
        X_domain_scaled = scaler_domain.fit_transform(X_domain_dense)

        # --- UMAP ---
        print("  Applying UMAP...")
        # Ensure verbose is handled correctly (might be passed via umap_params)
        # We'll set verbose=True during fit for progress, then restore original setting
        verbose_setting = umap_params.get('verbose', False)
        current_umap_params = umap_params.copy() # Don't modify the default dict
        current_umap_params['verbose'] = True # Show progress during fit

        reducer_domain = umap.UMAP(**current_umap_params)

        X_domain_reduced_umap = reducer_domain.fit_transform(X_domain_scaled)

        # Restore original verbose setting if needed for the object itself
        reducer_domain.verbose = verbose_setting

        print(f"  UMAP complete. Reduced shape: {X_domain_reduced_umap.shape}")
        return X_domain_reduced_umap, scaler_domain, reducer_domain

    except Exception as e:
        print(f"  Error during scaling or UMAP: {e}")
        return None

# --- Clustering Functions ---
# K-Means
def evaluate_k(
    X_reduced_umap: np.ndarray,
    domain_name: str,
    k_range: range = DEFAULT_K_RANGE,
    sample_size: int = DEFAULT_SILHOUETTE_SAMPLE_SIZE,
    kmeans_params: Dict = DEFAULT_KMEANS_PARAMS
) -> Optional[int]:
    """
    Calculates and plots Elbow and Silhouette scores to help choose k.

    Args:
        X_reduced_umap: The UMAP-reduced feature matrix for the domain.
        domain_name: Name of the domain for plot titles.
        k_range: Range of k values to test.
        sample_size: Sample size for Silhouette calculation.
        kmeans_params: Dictionary of base parameters for KMeans.

    Returns:
        The suggested optimal k based on the highest Silhouette score,
        or the smallest k if scores cannot be calculated. Returns None on error.
    """
    print("\n--- Determining Optimal k ---")
    if X_reduced_umap is None or X_reduced_umap.shape[0] < 2:
        print("  Error: Reduced feature matrix is empty or has less than 2 samples.")
        return None

    inertia_domain = []
    silhouette_scores_domain = []
    k_values_domain = list(k_range)
    actual_sample_size = min(sample_size, X_reduced_umap.shape[0])

    # Prepare sample for Silhouette
    if actual_sample_size > 1:
        # Use the random_state from kmeans_params if available, otherwise global RANDOM_STATE
        sampling_random_state = kmeans_params.get('random_state', RANDOM_STATE)
        X_sample = resample(X_reduced_umap, n_samples=actual_sample_size, random_state=sampling_random_state, replace=False)
        print(f"  Using sample size {actual_sample_size} for Silhouette score.")
    else:
        X_sample = X_reduced_umap # Use all data if too small
        print("  Warning: Dataset too small for sampling, using all data for Silhouette.")

    print("  Calculating Inertia and Silhouette scores...")
    for k in k_values_domain:
        # Create KMeans instance with combined default and specific k
        current_kmeans_params = {**kmeans_params, 'n_clusters': k}
        kmeans_eval = KMeans(**current_kmeans_params)

        # Elbow (fit on full reduced data)
        kmeans_eval.fit(X_reduced_umap)
        inertia_domain.append(kmeans_eval.inertia_)

        # Silhouette (predict on sample using model fitted on full data)
        if actual_sample_size > 1:
            cluster_labels_sample = kmeans_eval.predict(X_sample)
            if len(np.unique(cluster_labels_sample)) > 1: # Score requires at least 2 labels
                try:
                    score = silhouette_score(X_sample, cluster_labels_sample)
                    silhouette_scores_domain.append(score)
                except Exception as sil_e:
                    print(f"    Warning: Error calculating Silhouette for k={k}: {sil_e}")
                    silhouette_scores_domain.append(-1) # Invalid score
            else:
                # Only 1 cluster label found in the sample prediction
                silhouette_scores_domain.append(-1) # Invalid score
        else:
             # Cannot calculate Silhouette if sample size <= 1
             silhouette_scores_domain.append(-1)

    # --- Plotting ---
    fig, ax1 = plt.subplots(figsize=(25, 8))
    color = 'tab:blue'
    ax1.set_xlabel('Number of clusters (k)')
    ax1.set_ylabel('Inertia', color=color)
    ax1.plot(k_values_domain, inertia_domain, marker='o', color=color, label='Inertia')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.set_xticks(k_values_domain)
    ax1.grid(True)
    plt.xticks(rotation=90)

    # Check if silhouette_scores_domain has valid scores
    valid_indices = [i for i, s in enumerate(silhouette_scores_domain) if s > -1]
    suggested_k = k_values_domain[0]

    if valid_indices:
        valid_k_domain = [k_values_domain[i] for i in valid_indices]
        valid_scores_domain = [silhouette_scores_domain[i] for i in valid_indices]

        ax2 = ax1.twinx()
        color = 'tab:red'
        ax2.set_ylabel('Avg Silhouette Score (Sample)', color=color)
        ax2.plot(valid_k_domain, valid_scores_domain, marker='x', linestyle='--', color=color, label='Silhouette (Sample)')
        ax2.tick_params(axis='y', labelcolor=color)

        # Suggest best k based on Silhouette
        best_k_silhouette_idx_in_valid = np.argmax(valid_scores_domain)
        suggested_k = valid_k_domain[best_k_silhouette_idx_in_valid]
        print(f"  Suggested k = {suggested_k} (Max Silhouette Score: {valid_scores_domain[best_k_silhouette_idx_in_valid]:.4f})")

        # Combine legends
        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels + labels2, loc='center right')
    else:
        print("  Could not calculate valid Silhouette scores.")
        ax1.legend(loc='upper right')


    plt.title(f'Elbow & Silhouette Analysis for Domain: {domain_name}')
    fig.tight_layout()
    plt.show()

    return suggested_k

def apply_clustering(
    X_reduced_umap: np.ndarray,
    chosen_k: int,
    df: pd.DataFrame,
    df_domain_index: pd.Index,
    domain_name: str,
    kmeans_params: Dict = DEFAULT_KMEANS_PARAMS
) -> Optional[Tuple[pd.DataFrame, KMeans]]:
    """
    Applies K-Means with the chosen k and adds sub-cluster labels to the main DataFrame.

    Args:
        X_reduced_umap: The UMAP-reduced feature matrix for the domain.
        chosen_k: The selected number of clusters.
        df: The main DataFrame (will be modified).
        df_domain_index: The index of the rows belonging to the current domain in the main df.
        domain_name: Name of the domain for context.
        kmeans_params: Dictionary of base parameters for KMeans.

    Returns:
        A tuple containing:
        - df: The modified main DataFrame with 'sub_cluster' labels added/updated.
        - kmeans_model: The fitted KMeans object.
        Returns None on error.
    """
    print(f"\n--- Applying K-Means with k={chosen_k} for Domain: {domain_name} ---")
    if X_reduced_umap is None or chosen_k < 2:
        print("  Error: Invalid input data or chosen_k < 2.")
        return None

    try:
        # Create KMeans instance
        current_kmeans_params = {**kmeans_params, 'n_clusters': chosen_k}
        kmeans_model = KMeans(**current_kmeans_params)

        # Fit and predict
        cluster_labels_domain = kmeans_model.fit_predict(X_reduced_umap)

        # Add labels back to the main DataFrame using the correct index
        # Ensure 'sub_cluster' column exists
        if 'sub_cluster' not in df.columns:
             # Initialize with a type that can hold NaNs and integers if needed
             df['sub_cluster'] = pd.Series(index=df.index, dtype='Int64') # Use nullable integer type

        # Assign labels using .loc - this modifies the main df
        df.loc[df_domain_index, 'sub_cluster'] = cluster_labels_domain

        print(f"  Added/Updated sub-cluster labels for '{domain_name}' in main DataFrame.")
        print(f"  Sub-cluster counts for {domain_name}:")
        # Count labels just assigned
        print(pd.Series(cluster_labels_domain).value_counts().sort_index())

        return df, kmeans_model

    except Exception as e:
        print(f"  Error during K-Means clustering or labeling: {e}")
        return None
# HDBSCAN
def apply_hdbscan_clustering(
    X_reduced_umap: np.ndarray,
    df: pd.DataFrame,
    df_domain_index: pd.Index,
    domain_name: str,
    min_cluster_size: int = 15,
    min_samples: Optional[int] = None, # If None, defaults to min_cluster_size
    metric: str = 'euclidean',
    cluster_selection_method: str = 'eom', # Excess of Mass
    allow_single_cluster: bool = False # HDBSCAN parameter
) -> Optional[Tuple[pd.DataFrame, hdbscan.HDBSCAN, int]]:
    """
    Applies HDBSCAN clustering and adds sub-cluster labels to the main DataFrame.
    Noise points are labeled as -1.

    Args:
        X_reduced_umap: The UMAP-reduced feature matrix for the domain.
        df: The main DataFrame (will be modified).
        df_domain_index: The index of the rows belonging to the current domain in the main df.
        domain_name: Name of the domain for context.
        min_cluster_size: The minimum size of clusters.
        min_samples: The number of samples in a neighborhood for a point
                     to be considered as a core point.
        metric: The metric to use when calculating distance between instances in a feature array.
        cluster_selection_method: The method used to select clusters from the condensed tree.
        allow_single_cluster: Whether to allow HDBSCAN to return a single cluster.

    Returns:
        A tuple containing:
        - df: The modified main DataFrame with 'sub_cluster' labels added/updated.
        - hdbscan_model: The fitted HDBSCAN object.
        - n_clusters_found: The number of clusters found (excluding noise).
        Returns None on error.
    """
    print(f"\n--- Applying HDBSCAN for Domain: {domain_name} ---")
    if X_reduced_umap is None or X_reduced_umap.shape[0] < 2 : # HDBSCAN needs at least 2 samples
        print("  Error: Invalid input data for HDBSCAN (must have at least 2 samples).")
        return None

    try:
        hdbscan_model = hdbscan.HDBSCAN(
            min_cluster_size=min_cluster_size,
            min_samples=min_samples,
            metric=metric,
            cluster_selection_method=cluster_selection_method,
            allow_single_cluster=allow_single_cluster,
            gen_min_span_tree=True # Useful for some diagnostics or alternative cluster extraction
        )

        cluster_labels_domain = hdbscan_model.fit_predict(X_reduced_umap)

        # Calculate number of clusters found (excluding noise label -1)
        unique_labels = set(cluster_labels_domain)
        n_clusters_found = len(unique_labels) - (1 if -1 in unique_labels else 0)
        
        print(f"  HDBSCAN found {n_clusters_found} cluster(s) and {np.sum(cluster_labels_domain == -1)} noise points.")

        if 'sub_cluster' not in df.columns:
             df['sub_cluster'] = pd.Series(index=df.index, dtype='Int64')

        df.loc[df_domain_index, 'sub_cluster'] = cluster_labels_domain

        print(f"  Added/Updated sub-cluster labels for '{domain_name}' in main DataFrame.")
        print(f"  Sub-cluster counts for {domain_name} (HDBSCAN):")
        print(pd.Series(cluster_labels_domain).value_counts().sort_index())

        return df, hdbscan_model, n_clusters_found

    except Exception as e:
        print(f"  Error during HDBSCAN clustering or labeling: {e}")
        return None
   

# --- Function to Identify Emerging Clusters ---
from pandas.tseries.offsets import DateOffset

def identify_emerging_cluster_ids(
    df: pd.DataFrame,
    domain_name: str,
    cluster_column: str = 'sub_cluster',
    date_column: str = 'first_date',
    recent_months_window: int = 12,
    min_papers_recent_period: int = 1, # Min papers for a cluster in recent period to be considered
    emerging_ratio_threshold: float = 1.5, # Ratio of recent_prop/baseline_prop
    emerging_diff_threshold: float = 0.01, # Absolute increase in proportion (e.g., 0.01 = 1% increase)
    newly_active_min_recent_prop: float = 0.001 # Min proportion in recent period to be "newly active"
) -> List[int]:
    """
    Identifies emerging cluster IDs within a specific domain by comparing their
    proportion of publications in a recent period versus a baseline period.
    The computation is done directly on the input DataFrame.

    Args:
        df (pd.DataFrame): The main DataFrame containing paper data, including
                           domain boolean columns, cluster labels, and dates.
        domain_name (str): The name of the boolean column representing the domain.
        cluster_column (str): The name of the column containing cluster labels (defaults to 'sub_cluster').
        date_column (str): The name of the column containing publication dates.
        recent_months_window (int): The number of months to consider for the "recent" period.
        min_papers_recent_period (int): Minimum number of papers a cluster must have
                                        in the recent period to be considered potentially emerging.
        emerging_ratio_threshold (float): Minimum ratio of (recent_prop / baseline_prop)
                                          for a cluster to be considered emerging (if baseline > 0).
        emerging_diff_threshold (float): Minimum absolute increase in proportion
                                         (recent_prop - baseline_prop) for a cluster to be
                                         considered emerging.
        newly_active_min_recent_prop (float): Minimum proportion in the recent period for a
                                              cluster with no baseline presence to be considered "newly active".

    Returns:
        List[int]: A list of cluster IDs identified as emerging or newly active.
                   Returns an empty list if data is insufficient or errors occur.
    """
    print(f"\n--- Identifying Emerging Cluster IDs for Domain: {domain_name} ---")
    emerging_cluster_ids: List[int] = []

    # --- 1. Input Validation and Data Preparation ---
    if domain_name not in df.columns:
        print(f"  Error: Domain column '{domain_name}' not found.")
        return emerging_cluster_ids
    if cluster_column not in df.columns:
        print(f"  Error: Cluster column '{cluster_column}' not found.")
        return emerging_cluster_ids
    if date_column not in df.columns:
        print(f"  Error: Date column '{date_column}' not found.")
        return emerging_cluster_ids

    # Filter for the domain and operate on a copy
    df_domain_analysis = df[df[domain_name] == True].copy()

    if df_domain_analysis.empty:
        print(f"  No data found for domain '{domain_name}'.")
        return emerging_cluster_ids

    if not pd.api.types.is_datetime64_any_dtype(df_domain_analysis[date_column]):
        try:
            df_domain_analysis.loc[:, date_column] = pd.to_datetime(df_domain_analysis[date_column])
        except Exception as e:
            print(f"  Error converting date column '{date_column}' to datetime: {e}")
            return emerging_cluster_ids

    # Drop rows where cluster label is NA and ensure integer type
    df_domain_analysis.dropna(subset=[cluster_column], inplace=True)
    try:
        df_domain_analysis.loc[:, cluster_column] = df_domain_analysis[cluster_column].astype(int)
        # Filter out noise cluster if it's labeled as -1 (common for HDBSCAN)
        # For K-Means, this usually isn't an issue unless -1 is a valid label.
        df_domain_analysis = df_domain_analysis[df_domain_analysis[cluster_column] != -1]
    except ValueError:
        print(f"  Warning: Could not convert cluster column '{cluster_column}' to int. Proceeding with original type.")

    if df_domain_analysis.empty:
        print(f"  No valid clustered data found for domain '{domain_name}' after filtering noise/NA.")
        return emerging_cluster_ids

    # --- 2. Define Time Windows ---
    max_date_in_domain = df_domain_analysis[date_column].max()
    if pd.isna(max_date_in_domain):
        print("  Error: No valid maximum date found in the domain's data.")
        return emerging_cluster_ids

    recent_period_start_date = (max_date_in_domain - DateOffset(months=recent_months_window - 1)).replace(day=1, hour=0, minute=0, second=0, microsecond=0)

    print(f"  Most recent date in domain: {max_date_in_domain.strftime('%Y-%m-%d')}")
    print(f"  Recent period starts: {recent_period_start_date.strftime('%Y-%m-%d')} (approx. last {recent_months_window} months)")

    df_recent = df_domain_analysis[df_domain_analysis[date_column] >= recent_period_start_date]
    df_baseline = df_domain_analysis[df_domain_analysis[date_column] < recent_period_start_date]

    if df_recent.empty:
        print(f"  No data found in the recent {recent_months_window}-month period for domain '{domain_name}'. Cannot assess emergence.")
        return emerging_cluster_ids

    # --- 3. Calculate Proportions and Identify Emerging Clusters ---
    total_papers_domain_recent = len(df_recent)
    total_papers_domain_baseline = len(df_baseline)

    print(f"  Total papers in domain - Recent: {total_papers_domain_recent}, Baseline: {total_papers_domain_baseline}")

    unique_clusters = sorted(df_domain_analysis[cluster_column].unique())

    for cluster_id in unique_clusters:
        papers_cluster_recent = len(df_recent[df_recent[cluster_column] == cluster_id])
        
        # Skip if cluster has too few papers in the recent period
        if papers_cluster_recent < min_papers_recent_period:
            continue

        papers_cluster_baseline = len(df_baseline[df_baseline[cluster_column] == cluster_id])

        prop_recent = (papers_cluster_recent / total_papers_domain_recent) if total_papers_domain_recent > 0 else 0
        prop_baseline = (papers_cluster_baseline / total_papers_domain_baseline) if total_papers_domain_baseline > 0 else 0
        
        is_emerging = False

        if prop_baseline == 0: # Cluster is new or was not present in baseline
            if prop_recent >= newly_active_min_recent_prop:
                print(f"    Cluster {cluster_id}: Newly Active (Recent Prop: {prop_recent:.4f})")
                is_emerging = True
        else: # Cluster existed in baseline
            emergence_ratio = prop_recent / prop_baseline
            emergence_difference = prop_recent - prop_baseline

            if emergence_ratio >= emerging_ratio_threshold and emergence_difference >= emerging_diff_threshold:
                print(f"    Cluster {cluster_id}: Emerging (Ratio: {emergence_ratio:.2f}, Diff: {emergence_difference:.4f}, Recent Prop: {prop_recent:.4f}, Baseline Prop: {prop_baseline:.4f})")
                is_emerging = True
        
        if is_emerging:
            emerging_cluster_ids.append(cluster_id)

    if not emerging_cluster_ids:
        print("  No clusters met the criteria for being 'emerging' or 'newly active'.")
    else:
        print(f"  Identified emerging/newly active cluster IDs: {emerging_cluster_ids}")
        
    return sorted(list(set(emerging_cluster_ids))) # Return unique sorted list



In [ ]:
# --- Functions to Analyze Clusters and Plot Trends ---
def analyze_sub_clusters(
    df: pd.DataFrame,
    domain_name: str,
    chosen_k: int,
    relevant_features_used_for_clustering: List[str],
    metadata_features: List[str],
    rolling_window_param = ROLLING_WINDOW
) -> Optional[Tuple[pd.DataFrame, Dict[int, List[str]]]]:
    """
    Analyzes sub-clusters. For each sub-cluster, plots:
    1. Faceted bar plot of its Top 5 keywords (by mean proportion within the cluster).
       (Top 10 keywords are generated and stored for other uses).
    2. Faceted bar plot of its Top 5 unique areas (by mean proportion within the cluster).
    3. Faceted line plot of its relative temporal trend (proportion of monthly papers).
    Returns smoothed proportions DataFrame for temporal trends and a dictionary of top N keywords per sub-cluster.
    """
    print(f"\n--- Analyzing Sub-Clusters for {domain_name} (k={chosen_k} actual clusters) ---")
    smoothed_proportions_df_to_return = None
    top_keywords_map: Dict[int, List[str]] = {} # Stores top N (e.g., 10) keywords

    N_TOP_KEYWORDS_TO_GENERATE = 10 # How many keywords to identify and store in top_keywords_map
    N_KEYWORDS_TO_PLOT = 5        # How many keywords to display in the bar plot

    # --- Filtering and Grouping ---
    try:
        # Filter for the current domain and ensure sub_cluster labels are present
        # This mask will include papers belonging to any cluster (0, 1, ..., and -1 if HDBSCAN noise)
        analysis_mask = (df.loc[:, domain_name] == True) & (df.loc[:, 'sub_cluster'].notna())
        df_analysis = df.loc[analysis_mask].copy()
    except KeyError as e:
        print(f"  Error accessing columns for filtering: {e}.")
        return None, None # Return tuple of Nones

    if not df_analysis.empty:
        try:
            # Convert sub_cluster to int. If HDBSCAN was used, -1 for noise is already int.
            # If K-Means was used, labels are already int.
            # This ensures consistency if the column was, e.g., float.
            df_analysis['sub_cluster'] = df_analysis['sub_cluster'].astype(int)
        except Exception as e:
            print(f"  Warning: Could not convert 'sub_cluster' to int: {e}")
            # Proceeding, but downstream functions might expect int.

    if df_analysis.empty:
        print("  No data with sub-cluster labels found for analysis in this domain.")
        return None, None # Return tuple of Nones

    try:
        grouped_sub_clusters = df_analysis.groupby('sub_cluster')
    except Exception as e:
         print(f"  Error grouping by 'sub_cluster': {e}")
         return None, None # Return tuple of Nones

    # --- Analyze and Plot Top Keywords per Sub-cluster ---
    print(f"\n  Generating Top {N_KEYWORDS_TO_PLOT} Keywords per Sub-cluster Plot")
    actual_keyword_cols_for_summary = [
        col for col in relevant_features_used_for_clustering if col.startswith('kw_') and col in df_analysis.columns
    ]
    keyword_plot_data_list = [] # Data for the bar plot (top 5)

    if not actual_keyword_cols_for_summary:
        print("    No 'kw_' prefixed keyword columns found for summary.")
    else:
        try:
            # Mean proportion of each keyword *within each cluster*
            sub_cluster_keyword_proportions = grouped_sub_clusters[actual_keyword_cols_for_summary].mean()

            # Iterate through each cluster ID found in the data (includes 0, 1,... and -1 if present)
            for sub_cluster_id in sub_cluster_keyword_proportions.index:
                cluster_keyword_means = sub_cluster_keyword_proportions.loc[sub_cluster_id]
                if isinstance(cluster_keyword_means.dtype, pd.SparseDtype):
                    cluster_keyword_means = cluster_keyword_means.sparse.to_dense()

                # Identify Top N (e.g., 10) keywords for this cluster
                top_n_generated_kws = cluster_keyword_means.sort_values(ascending=False).head(N_TOP_KEYWORDS_TO_GENERATE)
                # Store these Top N (e.g., 10) keywords in the map for external use
                top_keywords_map[sub_cluster_id] = [kw.replace('kw_', '') for kw in top_n_generated_kws.index.tolist()]

                # For plotting, select only the Top X (e.g., 5) from the generated top N
                top_x_kws_for_plot = top_n_generated_kws.head(N_KEYWORDS_TO_PLOT)

                for keyword, proportion in top_x_kws_for_plot.items():
                    keyword_plot_data_list.append({
                        'sub_cluster': sub_cluster_id,
                        'item_name': keyword.replace('kw_', ''),
                        'percentage': proportion * 100
                    })

            if keyword_plot_data_list:
                keyword_plot_df = pd.DataFrame(keyword_plot_data_list)
                # Sort facets by sub_cluster_id for consistent order if not already
                cluster_order_keywords = sorted(keyword_plot_df['sub_cluster'].unique())

                g_keywords = sns.FacetGrid(keyword_plot_df, row="sub_cluster", row_order=cluster_order_keywords,
                                           aspect=4, height=1.2,
                                           sharex=True, sharey=False)
                g_keywords.map_dataframe(sns.barplot, x="percentage", y="item_name", orient="h", palette="crest_r")

                for ax_idx, ax_row in enumerate(g_keywords.axes):
                    for c_idx, c in enumerate(ax_row[0].containers):
                        labels = [f'{w:.1f}%' for w in c.datavalues]
                        ax_row[0].bar_label(c, labels=labels, label_type='edge', padding=3, fontsize=8)

                g_keywords.set_titles(row_template=f"Sub-cluster {{row_name}} - Top {N_KEYWORDS_TO_PLOT} Keywords")
                g_keywords.set_axis_labels("Avg. Presence in Cluster Papers (%)", "Keyword")
                g_keywords.figure.subplots_adjust(top=0.93)
                g_keywords.figure.suptitle(f'Top {N_KEYWORDS_TO_PLOT} Keywords by Sub-cluster ({domain_name})', fontsize=16, y=1.0)
                plt.tight_layout(rect=[0,0,1,0.96])
                plt.show()
            else:
                print("    No keyword data to plot.")
        except Exception as e:
            print(f"    Error calculating or plotting top keywords: {e}")
            # Fall through to try other plots

    # --- Analyze and Plot Top 5 Unique Areas per Sub-cluster ---
    print(f"\n  Generating Top 5 Unique Areas per Sub-cluster Plot:")
    unique_area_cols = [col for col in metadata_features if col != 'number_of_authors' and col in df_analysis.columns]
    area_plot_data_list = []

    if not unique_area_cols:
        print("    No unique area columns identified for plotting.")
    else:
        try:
            area_proportions_per_cluster = grouped_sub_clusters[unique_area_cols].mean()
            for sub_cluster_id in area_proportions_per_cluster.index:
                cluster_area_means = area_proportions_per_cluster.loc[sub_cluster_id]
                top_5_areas_for_this_cluster = cluster_area_means.sort_values(ascending=False).head(5)
                for area, proportion in top_5_areas_for_this_cluster.items():
                    if proportion > 0.001: # Only plot if proportion is somewhat significant
                        area_plot_data_list.append({
                            'sub_cluster': sub_cluster_id,
                            'item_name': area,
                            'percentage': proportion * 100
                        })
            if area_plot_data_list:
                area_plot_df = pd.DataFrame(area_plot_data_list)
                cluster_order_areas = sorted(area_plot_df['sub_cluster'].unique())
                g_areas = sns.FacetGrid(area_plot_df, row="sub_cluster", row_order=cluster_order_areas,
                                        aspect=4, height=1.2,
                                        sharex=True, sharey=False)
                g_areas.map_dataframe(sns.barplot, x="percentage", y="item_name", orient="h", palette="flare_r")
                for ax_idx, ax_row in enumerate(g_areas.axes):
                    for c_idx, c in enumerate(ax_row[0].containers):
                        labels = [f'{w:.1f}%' for w in c.datavalues]
                        ax_row[0].bar_label(c, labels=labels, label_type='edge', padding=3, fontsize=8)
                g_areas.set_titles(row_template="Sub-cluster {row_name} - Top 5 Areas")
                g_areas.set_axis_labels("Avg. Presence in Cluster Papers (%)", "Unique Area")
                g_areas.figure.subplots_adjust(top=0.93)
                g_areas.figure.suptitle(f'Top 5 Unique Areas by Sub-cluster ({domain_name})', fontsize=16, y=1.0)
                plt.tight_layout(rect=[0,0,1,0.96])
                plt.show()
            else:
                print("    No area data to plot (or proportions too low).")
        except Exception as e:
            print(f"    Error plotting unique area distributions: {e}")
            # Fall through

    # --- Analyze Temporal Trends (Proportions Plot - Faceted) ---
    if 'first_date' in df_analysis.columns and pd.api.types.is_datetime64_any_dtype(df_analysis['first_date']):
        print("\n  Relative Temporal Trends (Proportion of Monthly Publications):")
        try:
            # Use .loc for assignment to avoid SettingWithCopyWarning
            df_analysis.loc[:, 'YearMonth'] = df_analysis['first_date'].dt.to_period('M')
            # Count papers per YearMonth and sub_cluster
            cluster_monthly_counts = df_analysis.groupby(['YearMonth', 'sub_cluster']).size().unstack(fill_value=0)
            # Total papers per YearMonth across all clusters (including -1 if present)
            total_monthly_counts = cluster_monthly_counts.sum(axis=1)
            # Calculate proportion
            proportions_df = cluster_monthly_counts.divide(total_monthly_counts, axis=0).fillna(0)

            # Smooth proportions
            smoothed_proportions_df_to_return = proportions_df.rolling(window=rolling_window_param, center=True, min_periods=1).mean()
            # Convert PeriodIndex to DatetimeIndex for plotting
            smoothed_proportions_df_to_return.index = smoothed_proportions_df_to_return.index.to_timestamp()

            # Melt for FacetGrid
            proportions_melted = smoothed_proportions_df_to_return.reset_index().melt(
                id_vars='YearMonth', var_name='sub_cluster', value_name='proportion'
            )
            # Ensure consistent cluster order in facets
            cluster_order_temporal = sorted(df_analysis['sub_cluster'].unique())

            g_temporal = sns.FacetGrid(
                proportions_melted, row="sub_cluster", row_order=cluster_order_temporal, hue="sub_cluster",
                hue_order=cluster_order_temporal, aspect=5, height=1.5,
                palette=sns.color_palette("hsv", n_colors=len(cluster_order_temporal)), # Use a palette with enough colors
                sharex=True, sharey=False # Allow different y-scales if proportions vary a lot
            )
            g_temporal.map(sns.lineplot, "YearMonth", "proportion", lw=2)
            g_temporal.set_titles(row_template="Sub-cluster {row_name}")
            g_temporal.set_axis_labels("Time (Month)", "Proportion of Monthly Papers")
            g_temporal.map(plt.grid, axis='y', linestyle='--', alpha=0.7)
            g_temporal.figure.subplots_adjust(top=0.93)
            g_temporal.figure.suptitle(f'Relative Trend within {domain_name} ({rolling_window_param}-Month Rolling Avg)', fontsize=16, y=1.0)
            plt.tight_layout(rect=[0,0,1,0.96])
            plt.show()

            # Clean up temporary column
            if 'YearMonth' in df_analysis.columns:
                df_analysis.drop(columns=['YearMonth'], inplace=True, errors='ignore')

        except Exception as plot_e:
            print(f"    Error during proportion plot generation: {plot_e}")
            if 'YearMonth' in df_analysis.columns:
                df_analysis.drop(columns=['YearMonth'], inplace=True, errors='ignore')
            # Still return what we have, top_keywords_map should be populated by now
            return smoothed_proportions_df_to_return, top_keywords_map

        print("\n  Median Publication Date per Sub-cluster:")
        try:
            median_dates = grouped_sub_clusters['first_date'].median()
            print(median_dates)
        except Exception as e:
            print(f"    Error calculating median dates: {e}")

        # top_keywords_map will contain the top N_TOP_KEYWORDS_TO_GENERATE keywords
        return smoothed_proportions_df_to_return, top_keywords_map
    else:
        print("\n  Skipping temporal analysis (missing/invalid 'first_date').")
        # top_keywords_map will contain the top N_TOP_KEYWORDS_TO_GENERATE keywords
        return smoothed_proportions_df_to_return, top_keywords_map

def plot_combined_trends_legend(
    proportions_df: pd.DataFrame,
    clusters_to_plot: List[int],
    cluster_colors: Dict[int, str],
    top_keywords_data: Dict[int, List[str]],
    domain_name: str,
    rolling_window: int = 6,
    keyword_exclusion_color: str = 'grey',
    annotation_fontsize: int = 9,
    annotation_start_y: float = 0.97,
    annotation_y_step: float = 0.18,
    y_axis_scale: str = 'linear',  # 'linear' or 'log'
    display_excluded_clusters: bool = True  # Toggle for displaying grey lines
):
    """
    Plots smoothed proportion trends (as percentages) for selected sub-clusters on a single graph
    with custom colors and keyword annotations on the right.

    Args:
        proportions_df: DataFrame of smoothed proportions (values between 0 and 1).
        clusters_to_plot: List of cluster IDs to plot.
        cluster_colors: Dictionary mapping cluster IDs to their respective colors.
        top_keywords_data: Dictionary mapping cluster IDs to their top keywords.
        domain_name: Name of the domain for the plot title.
        rolling_window: Rolling window size for smoothing.
        keyword_exclusion_color: Color used for excluded clusters (default: 'grey').
        annotation_fontsize: Font size for keyword annotations.
        annotation_start_y: Starting y-position for keyword annotations.
        annotation_y_step: Step size for y-position of annotations.
        y_axis_scale: Scale of the y-axis. Can be 'linear' or 'log'. Defaults to 'linear'.
        display_excluded_clusters: Whether to display excluded clusters (grey lines). Defaults to True.
    """
    print(f"\n--- Plotting Combined Trends for Selected Sub-clusters in {domain_name} (Y-axis: {y_axis_scale}) ---")

    if proportions_df is None or proportions_df.empty:
        print("  Error: Input proportions_df is None or empty.")
        return

    # Identify clusters to include and exclude
    clusters_present = [c for c in clusters_to_plot if c in proportions_df.columns]
    missing_clusters = [c for c in clusters_to_plot if c not in proportions_df.columns]
    if missing_clusters:
        print(f"  Warning: Requested clusters not in proportions_df: {missing_clusters}")
    if not clusters_present:
        print("  Error: None of the requested clusters found in proportions_df.")
        return

    df_to_plot = proportions_df[clusters_present].copy()

    # Separate clusters by color
    included_clusters = [c for c in clusters_present if cluster_colors.get(c) != keyword_exclusion_color]
    excluded_clusters = [c for c in clusters_present if cluster_colors.get(c) == keyword_exclusion_color]

    # Create a color palette for the clusters
    plot_palette = {}
    default_color_palette = sns.color_palette("tab10", n_colors=len(clusters_present))
    for i, cluster_id in enumerate(clusters_present):
        plot_palette[cluster_id] = cluster_colors.get(cluster_id, default_color_palette[i % len(default_color_palette)])

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(16, 8))

    # Plot excluded clusters (grey lines) first, if enabled
    if display_excluded_clusters:
        for cluster_id in excluded_clusters:
            color = plot_palette.get(cluster_id)
            plot_data = df_to_plot[cluster_id] * 100  # Convert proportion to percentage
            ax.plot(df_to_plot.index, plot_data, color=color, lw=1.5, alpha=0.5, zorder=1)

    # Plot included clusters (colored lines)
    for cluster_id in included_clusters:
        color = plot_palette.get(cluster_id)
        plot_data = df_to_plot[cluster_id] * 100  # Convert proportion to percentage
        ax.plot(df_to_plot.index, plot_data, color=color, lw=2.5, zorder=2, label=f"Cluster {cluster_id}")

    # --- Axis Labels and Title ---
    ax.set_title(f'Relative Trend for Clusters within {domain_name}')
    ax.set_xlabel('Time')
    ax.set_ylabel('Percentage of Monthly Publications (%)')
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Set y-axis scale and formatter
    if y_axis_scale.lower() == 'log':
        ax.set_yscale('log')
        try:
            ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=100.0, decimals=None))
        except Exception:
            ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda y, _: '{:.1f}%'.format(y)))
    else:  # Linear scale
        ax.set_yscale('linear')
        ax.set_ylim(bottom=0, top=max(1.0, df_to_plot.mul(100).max().max() * 1.1) if not df_to_plot.empty else 1.0)
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=100.0, decimals=0))

    # Adjust layout and show plot
    plt.subplots_adjust(left=0.07, right=0.70, top=0.9, bottom=0.1)
    plt.legend(loc='upper left', bbox_to_anchor=(1.02, 1), title="Clusters")
    plt.show()

def plot_combined_trends(
    proportions_df: pd.DataFrame,
    clusters_to_plot: List[int],
    cluster_colors: Dict[int, str],
    top_keywords_data: Dict[int, List[str]],
    domain_name: str,
    rolling_window: int = 6,
    keyword_exclusion_color: str = 'grey',
    annotation_fontsize: int = 9,
    annotation_start_y: float = 0.97,
    annotation_y_step: float = 0.18,
    y_axis_scale: str = 'linear',  # 'linear' or 'log'
    max_keywords_to_display: int = 5  # New parameter to limit the number of keywords displayed
):
    """
    Plots smoothed proportion trends (as percentages) for selected sub-clusters on a single graph
    with custom colors and keyword annotations on the right.

    Args:
        proportions_df: DataFrame of smoothed proportions (values between 0 and 1).
        clusters_to_plot: List of cluster IDs to plot.
        cluster_colors: Dictionary mapping cluster IDs to their respective colors.
        top_keywords_data: Dictionary mapping cluster IDs to their top keywords.
        domain_name: Name of the domain for the plot title.
        rolling_window: Rolling window size for smoothing.
        keyword_exclusion_color: Color used for excluded clusters (default: 'grey').
        annotation_fontsize: Font size for keyword annotations.
        annotation_start_y: Starting y-position for keyword annotations.
        annotation_y_step: Step size for y-position of annotations.
        y_axis_scale: Scale of the y-axis. Can be 'linear' or 'log'. Defaults to 'linear'.
        max_keywords_to_display: Maximum number of keywords to display for each cluster. Defaults to 5.
    """
    print(f"\n--- Plotting Combined Trends for Selected Sub-clusters in {domain_name} (Y-axis: {y_axis_scale}) ---")

    if proportions_df is None or proportions_df.empty:
        print("  Error: Input proportions_df is None or empty.")
        return

    # Identify clusters to include and exclude
    clusters_present = [c for c in clusters_to_plot if c in proportions_df.columns]
    missing_clusters = [c for c in clusters_to_plot if c not in proportions_df.columns]
    if missing_clusters:
        print(f"  Warning: Requested clusters not in proportions_df: {missing_clusters}")
    if not clusters_present:
        print("  Error: None of the requested clusters found in proportions_df.")
        return

    df_to_plot = proportions_df[clusters_present].copy()

    # Separate clusters by color
    included_clusters = [c for c in clusters_present if cluster_colors.get(c) != keyword_exclusion_color]
    excluded_clusters = [c for c in clusters_present if cluster_colors.get(c) == keyword_exclusion_color]

    # Create a color palette for the clusters
    plot_palette = {}
    default_color_palette = sns.color_palette("tab10", n_colors=len(clusters_present))
    for i, cluster_id in enumerate(clusters_present):
        plot_palette[cluster_id] = cluster_colors.get(cluster_id, default_color_palette[i % len(default_color_palette)])

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(16, 8))

    # Plot excluded clusters (grey lines) first, if enabled
    for cluster_id in excluded_clusters:
        color = plot_palette.get(cluster_id)
        plot_data = df_to_plot[cluster_id] * 100  # Convert proportion to percentage
        ax.plot(df_to_plot.index, plot_data, color=color, lw=1.5, alpha=0.5, zorder=1)

    # Plot included clusters (colored lines)
    for cluster_id in included_clusters:
        color = plot_palette.get(cluster_id)
        plot_data = df_to_plot[cluster_id] * 100  # Convert proportion to percentage
        ax.plot(df_to_plot.index, plot_data, color=color, lw=2.5, zorder=2)

    # --- Keyword Annotations ---
    last_proportions = {
        cid: df_to_plot[cid].iloc[-1]
        for cid in clusters_present
        if not df_to_plot[cid].empty and pd.notna(df_to_plot[cid].iloc[-1])
    }
    sorted_clusters_for_text = sorted(last_proportions, key=last_proportions.get, reverse=True)
    clusters_not_in_sorted = [c for c in clusters_present if c not in sorted_clusters_for_text]
    sorted_clusters_for_text.extend(clusters_not_in_sorted)
    current_y_text_position = annotation_start_y

    for cluster_id in sorted_clusters_for_text:
        color = plot_palette.get(cluster_id)
        skip_keywords = False
        if color:
            if isinstance(color, str) and color.lower() == keyword_exclusion_color.lower():
                skip_keywords = True
        if skip_keywords:
            continue
        keywords = top_keywords_data.get(cluster_id, [])
        if keywords:
            # Limit the number of keywords displayed
            keywords_to_display = keywords[:max_keywords_to_display]
            keyword_string = f"Cluster {cluster_id}:\n" + "\n".join(keywords_to_display)
            ax.text(1.02, current_y_text_position, keyword_string,
                    transform=ax.transAxes, fontsize=annotation_fontsize, color=color,
                    verticalalignment='top', horizontalalignment='left',
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec=color, alpha=0.75))
            current_y_text_position -= annotation_y_step

    # --- Axis Labels and Title ---
    ax.set_title(f'Relative Trend for Clusters within {domain_name}')
    ax.set_xlabel('Time')
    ax.set_ylabel('Percentage of Monthly Publications (%)')
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Set y-axis scale and formatter
    if y_axis_scale.lower() == 'log':
        ax.set_yscale('log')
        try:
            ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=100.0, decimals=None))
        except Exception:
            ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda y, _: '{:.1f}%'.format(y)))
    else:  # Linear scale
        ax.set_yscale('linear')
        ax.set_ylim(bottom=0, top=max(1.0, df_to_plot.mul(100).max().max() * 1.1) if not df_to_plot.empty else 1.0)
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=100.0, decimals=0))

    # Adjust layout and show plot
    plt.subplots_adjust(left=0.07, right=0.70, top=0.9, bottom=0.1)
    plt.show()


def plot_cluster_keyword_trends(
    df: pd.DataFrame,
    domain_name: str,
    cluster_id: int,
    all_cluster_keywords: List[str],
    num_keywords_to_plot: int = 10,
    date_column: str = 'first_date',
    text_column: str = 'abstract',
    cluster_column: str = 'sub_cluster',
    rolling_window: int = ROLLING_WINDOW,
    palette_name: str = 'tab10'
):
    """
    Plots the temporal trend of the top N keywords for a specific cluster.

    For each of the selected keywords, it calculates the proportion of papers
    within the given cluster, per month, that contain the keyword in their abstract.
    These proportions are then smoothed using a rolling average and plotted.

    Args:
        df (pd.DataFrame): The main DataFrame containing paper data.
        domain_name (str): The domain to filter for (e.g., 'Quantitative Biology').
        cluster_id (int): The specific sub-cluster ID to analyze.
        all_cluster_keywords (List[str]): A list of all top keywords identified for this cluster.
                                          The function will select the top 'num_keywords_to_plot' from this list.
        num_keywords_to_plot (int): The number of top keywords to plot from 'all_cluster_keywords'.
        date_column (str): Name of the column containing publication dates (datetime objects).
        text_column (str): Name of the column containing text for keyword search (e.g., 'abstract').
        cluster_column (str): Name of the column containing sub-cluster labels.
        rolling_window (int): The window size for the rolling average.
        palette_name (str): Name of the seaborn color palette to use for plotting lines.
    """
    print(f"\n--- Plotting Keyword Trends for Domain: {domain_name}, Cluster: {cluster_id} ---")

    # --- 1. Filter Data for the specific domain and cluster ---
    try:
        df_filtered = df[(df[domain_name] == True) & (df[cluster_column] == cluster_id)].copy()
    except KeyError as e:
        print(f"  Error: Column not found during filtering: {e}. Make sure domain and cluster columns exist.")
        return

    if df_filtered.empty:
        print(f"  No data found for domain '{domain_name}' and cluster '{cluster_id}'. Skipping plot.")
        return

    # --- 2. Prepare Dates ---
    if date_column not in df_filtered.columns:
        print(f"  Error: Date column '{date_column}' not found.")
        return
    if not pd.api.types.is_datetime64_any_dtype(df_filtered[date_column]):
        try:
            df_filtered.loc[:, date_column] = pd.to_datetime(df_filtered[date_column])
        except Exception as e:
            print(f"  Error converting date column '{date_column}' to datetime: {e}. Skipping.")
            return
    
    df_filtered.loc[:, 'YearMonth'] = df_filtered[date_column].dt.to_period('M')

    # --- 3. Select Keywords to Plot ---
    if not all_cluster_keywords:
        print(f"  No keywords provided for cluster {cluster_id}. Skipping plot.")
        return
    
    keywords_to_plot = all_cluster_keywords[:num_keywords_to_plot]
    if len(keywords_to_plot) < num_keywords_to_plot:
        print(f"  Warning: Fewer than {num_keywords_to_plot} keywords available. Plotting {len(keywords_to_plot)} keywords.")
    if not keywords_to_plot:
        print(f"  No keywords selected to plot for cluster {cluster_id}. Skipping plot.")
        return

    print(f"  Plotting top {len(keywords_to_plot)} keywords: {', '.join(keywords_to_plot)}")

    # --- 4. Calculate Monthly Proportions for each Keyword ---
    keyword_monthly_proportions_dict = {}
    
    # Group once by YearMonth to optimize
    grouped_by_month = df_filtered.groupby('YearMonth')

    for keyword in keywords_to_plot:
        monthly_proportions = []
        # Ensure the keyword string is valid for regex
        safe_keyword_pattern = re.escape(keyword.lower())

        for ym, month_group in grouped_by_month:
            if text_column not in month_group.columns:
                print(f"  Error: Text column '{text_column}' not found in month_group for {ym}.")
                continue

            # Count papers containing the keyword (case-insensitive)
            papers_with_keyword = month_group[text_column].str.lower().str.contains(safe_keyword_pattern, na=False).sum()
            total_papers_this_month_in_cluster = len(month_group)
            
            proportion = papers_with_keyword / total_papers_this_month_in_cluster if total_papers_this_month_in_cluster > 0 else 0
            monthly_proportions.append({'YearMonth': ym, 'proportion': proportion})
        
        if monthly_proportions:
            keyword_ts = pd.DataFrame(monthly_proportions).set_index('YearMonth')['proportion']
            keyword_monthly_proportions_dict[keyword] = keyword_ts

    if not keyword_monthly_proportions_dict:
        print("  No keyword data could be processed for plotting.")
        return

    proportions_df = pd.DataFrame(keyword_monthly_proportions_dict)

    # --- 5. Handle Missing Months & Smooth Data ---
    # Create a full date range from min to max observed YearMonth
    if not proportions_df.empty:
        min_date = proportions_df.index.min().to_timestamp()
        max_date = proportions_df.index.max().to_timestamp()
        full_date_range = pd.period_range(start=min_date, end=max_date, freq='M')
        proportions_df = proportions_df.reindex(full_date_range, fill_value=0)
        
        # Apply rolling mean
        smoothed_proportions_df = proportions_df.rolling(window=rolling_window, center=True, min_periods=1).mean()
        # Convert PeriodIndex to DatetimeIndex for plotting
        smoothed_proportions_df.index = smoothed_proportions_df.index.to_timestamp()
    else:
        print("  Proportions DataFrame is empty before smoothing. Skipping plot.")
        return

    # --- 6. Plot ---
    if smoothed_proportions_df.empty:
        print("  Smoothed proportions DataFrame is empty. Skipping plot.")
        return

    plt.style.use('seaborn-v0_8-whitegrid') # Using a seaborn style
    fig, ax = plt.subplots(figsize=(15, 7))
    
    colors = sns.color_palette(palette_name, n_colors=len(smoothed_proportions_df.columns))

    for i, keyword in enumerate(smoothed_proportions_df.columns):
        ax.plot(smoothed_proportions_df.index, smoothed_proportions_df[keyword] * 100, # Plot as percentage
                label=keyword, color=colors[i % len(colors)], linewidth=2)

    ax.set_title(f'Temporal Trend of Top {len(keywords_to_plot)} Keywords in {domain_name} - Cluster {cluster_id}\n({rolling_window}-Month Rolling Average)', fontsize=16)
    ax.set_xlabel('Time (Month)', fontsize=12)
    ax.set_ylabel('Proportion of Cluster Papers Containing Keyword (%)', fontsize=12)
    
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=100.0, decimals=1))
    ax.legend(title='Keywords', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)
    
    plt.xticks(rotation=45)
    plt.tight_layout(rect=[0, 0, 0.85, 1]) # Adjust layout to make space for legend
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

def plot_cluster_keyword_trends_plotly(
    df: pd.DataFrame,
    domain_name: str,
    cluster_id: int,
    all_cluster_keywords: List[str],
    num_keywords_to_plot: int = 10,
    custom_colors: Optional[List[str]] = None, # New argument for custom hex colors
    date_column: str = 'first_date',
    text_column: str = 'abstract',
    cluster_column: str = 'sub_cluster',
    rolling_window: int = ROLLING_WINDOW
):
    """
    Plots the temporal trend of the top N keywords for a specific cluster using Plotly.

    For each of the selected keywords, it calculates the proportion of papers
    within the given cluster, per month, that contain the keyword in their abstract.
    These proportions are then smoothed using a rolling average and plotted.

    Args:
        df (pd.DataFrame): The main DataFrame containing paper data.
        domain_name (str): The domain to filter for (e.g., 'Quantitative Biology').
        cluster_id (int): The specific sub-cluster ID to analyze.
        all_cluster_keywords (List[str]): A list of all top keywords identified for this cluster.
                                          The function will select the top 'num_keywords_to_plot' from this list.
        num_keywords_to_plot (int): The number of top keywords to plot from 'all_cluster_keywords'.
        custom_colors (Optional[List[str]]): A list of hex color codes to use for plotting lines.
                                             If None, Plotly's default colors will be used.
                                             The list should have at least num_keywords_to_plot elements.
        date_column (str): Name of the column containing publication dates (datetime objects).
        text_column (str): Name of the column containing text for keyword search (e.g., 'abstract').
        cluster_column (str): Name of the column containing sub-cluster labels.
        rolling_window (int): The window size for the rolling average.
    """
    print(f"\n--- Plotting Keyword Trends (Plotly) for Domain: {domain_name}, Cluster: {cluster_id} ---")

    # --- 1. Filter Data for the specific domain and cluster ---
    try:
        df_filtered = df[(df[domain_name] == True) & (df[cluster_column] == cluster_id)].copy()
    except KeyError as e:
        print(f"  Error: Column not found during filtering: {e}. Make sure domain and cluster columns exist.")
        return

    if df_filtered.empty:
        print(f"  No data found for domain '{domain_name}' and cluster '{cluster_id}'. Skipping plot.")
        return

    # --- 2. Prepare Dates ---
    if date_column not in df_filtered.columns:
        print(f"  Error: Date column '{date_column}' not found.")
        return
    if not pd.api.types.is_datetime64_any_dtype(df_filtered[date_column]):
        try:
            df_filtered.loc[:, date_column] = pd.to_datetime(df_filtered[date_column])
        except Exception as e:
            print(f"  Error converting date column '{date_column}' to datetime: {e}. Skipping.")
            return

    df_filtered.loc[:, 'YearMonth'] = df_filtered[date_column].dt.to_period('M')

    # --- 3. Select Keywords to Plot ---
    if not all_cluster_keywords:
        print(f"  No keywords provided for cluster {cluster_id}. Skipping plot.")
        return

    keywords_to_plot = all_cluster_keywords[:num_keywords_to_plot]
    if len(keywords_to_plot) < num_keywords_to_plot:
        print(f"  Warning: Fewer than {num_keywords_to_plot} keywords available. Plotting {len(keywords_to_plot)} keywords.")
    if not keywords_to_plot:
        print(f"  No keywords selected to plot for cluster {cluster_id}. Skipping plot.")
        return

    print(f"  Plotting top {len(keywords_to_plot)} keywords: {', '.join(keywords_to_plot)}")

    if custom_colors and len(custom_colors) < len(keywords_to_plot):
        print(f"  Warning: Not enough custom colors provided ({len(custom_colors)}) for the number of keywords to plot ({len(keywords_to_plot)}). Plotly defaults will be used for some lines.")
        # Allow Plotly to cycle if not enough colors are provided, or extend with defaults.
        # For simplicity, we'll let Plotly handle it if the list is too short.

    # --- 4. Calculate Monthly Proportions for each Keyword ---
    keyword_monthly_proportions_dict = {}
    grouped_by_month = df_filtered.groupby('YearMonth')

    for keyword in keywords_to_plot:
        monthly_proportions = []
        safe_keyword_pattern = re.escape(keyword.lower())

        for ym, month_group in grouped_by_month:
            if text_column not in month_group.columns:
                print(f"  Error: Text column '{text_column}' not found in month_group for {ym}.")
                continue

            papers_with_keyword = month_group[text_column].str.lower().str.contains(safe_keyword_pattern, na=False).sum()
            total_papers_this_month_in_cluster = len(month_group)

            proportion = papers_with_keyword / total_papers_this_month_in_cluster if total_papers_this_month_in_cluster > 0 else 0
            monthly_proportions.append({'YearMonth': ym, 'proportion': proportion})

        if monthly_proportions:
            keyword_ts = pd.DataFrame(monthly_proportions).set_index('YearMonth')['proportion']
            keyword_monthly_proportions_dict[keyword] = keyword_ts

    if not keyword_monthly_proportions_dict:
        print("  No keyword data could be processed for plotting.")
        return

    proportions_df = pd.DataFrame(keyword_monthly_proportions_dict)

    # --- 5. Handle Missing Months & Smooth Data ---
    if proportions_df.empty:
        print("  Proportions DataFrame is empty before smoothing. Skipping plot.")
        return

    min_date_period = proportions_df.index.min()
    max_date_period = proportions_df.index.max()

    if pd.isna(min_date_period) or pd.isna(max_date_period):
        print("  Could not determine date range from proportions_df index. Skipping plot.")
        return

    full_date_range = pd.period_range(start=min_date_period.to_timestamp(),
                                      end=max_date_period.to_timestamp(), freq='M')
    proportions_df = proportions_df.reindex(full_date_range, fill_value=0)

    smoothed_proportions_df = proportions_df.rolling(window=rolling_window, center=True, min_periods=1).mean()
    smoothed_proportions_df.index = smoothed_proportions_df.index.to_timestamp() # Convert PeriodIndex to DatetimeIndex for Plotly

    # --- 6. Plot with Plotly ---
    if smoothed_proportions_df.empty:
        print("  Smoothed proportions DataFrame is empty. Skipping plot.")
        return

    fig = go.Figure()

    for i, keyword in enumerate(smoothed_proportions_df.columns):
        color = custom_colors[i % len(custom_colors)] if custom_colors else None
        fig.add_trace(go.Scatter(
            x=smoothed_proportions_df.index,
            y=smoothed_proportions_df[keyword] * 100, # Plot as percentage
            mode='lines',
            name=keyword,
            line=dict(width=2, color=color) # Apply custom color if provided
        ))

    fig.update_layout(
        title=f'Temporal Trend of Top {len(keywords_to_plot)} Keywords in {domain_name} - Cluster {cluster_id}<br>({rolling_window}-Month Rolling Average)',
        title_x=0.5,
        xaxis_title='Time (Month)',
        yaxis_title='Proportion of Cluster Papers Containing Keyword (%)',
        yaxis_tickformat='.1f', # Format y-axis ticks to one decimal place
        yaxis_ticksuffix='%',
        legend_title_text='Keywords',
        height=600,
        width=1000,
        margin=dict(l=50, r=50, b=100, t=100, pad=4), # Adjust margins
        hovermode='x unified' # Improved hover experience
    )
    fig.update_xaxes(tickangle=45)

    fig.show()


def plot_cluster_record_counts(
    df: pd.DataFrame,
    cluster_column: str,
    domain_name: Optional[str] = None,
    ax: Optional[plt.Axes] = None,
    top_n: Optional[int] = None
) -> None:
    """
    Generates and displays a bar plot showing the number of records in each actual cluster
    (excluding noise cluster -1) for a specified domain.
    Optionally shows only the top N clusters.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        cluster_column (str): The name of the column containing cluster labels.
        domain_name (Optional[str]): The name of the domain to filter by.
                                     If None, data is not filtered by domain.
                                     The domain_name column in df must be boolean.
        ax (Optional[plt.Axes]): A matplotlib axes object to plot on.
                                 If None, a new figure and axes will be created.
        top_n (Optional[int]): If set, displays only the top N largest clusters
                               and groups the rest into an 'Other' category.
    """
    if cluster_column not in df.columns:
        print(f"Error: Cluster column '{cluster_column}' not found in DataFrame.")
        return

    df_to_plot = df.copy()
    plot_title = "Number of Records per Actual Cluster"

    if domain_name:
        if domain_name not in df.columns:
            print(f"Warning: Domain column '{domain_name}' not found in DataFrame. Plotting for all data in df.")
        elif not pd.api.types.is_bool_dtype(df_to_plot[domain_name]):
            print(f"Warning: Domain column '{domain_name}' is not boolean. Plotting for all data in df.")
        else:
            df_to_plot = df_to_plot[df_to_plot[domain_name] == True]
            plot_title = f"Number of Records per Actual Cluster in {domain_name}"
    
    if df_to_plot.empty:
        if domain_name:
            print(f"No data to plot for domain '{domain_name}'.")
        else:
            print("No data to plot (DataFrame is empty).")
        return

    # Exclude noise cluster (-1)
    actual_clusters_df = df_to_plot[df_to_plot[cluster_column] != -1]
    
    if actual_clusters_df.empty:
        print(f"No actual (non-noise) cluster data found in '{cluster_column}' for the specified selection.")
        return
        
    cluster_counts = actual_clusters_df[cluster_column].value_counts()

    if top_n and len(cluster_counts) > top_n:
        top_clusters = cluster_counts.nlargest(top_n)
        other_count = cluster_counts.nsmallest(len(cluster_counts) - top_n).sum()
        if other_count > 0:
            top_clusters['Other (Remaining Clusters)'] = other_count
        cluster_counts = top_clusters
        plot_title += f" (Top {top_n})"
    
    cluster_counts = cluster_counts.sort_index(ascending=True, key=lambda x: pd.Series(x).astype(str) if 'Other' not in x.name else 'ZZZ') # Sort numerically, 'Other' last


    if cluster_counts.empty:
        print(f"No actual cluster data to plot after filtering and top_n in '{cluster_column}'.")
        return

    current_ax = ax
    if current_ax is None:
        # Adjust figsize based on number of clusters to plot
        num_items_to_plot = len(cluster_counts)
        fig_width = max(12, num_items_to_plot * 0.5) # Dynamic width
        fig_height = 7
        fig, current_ax = plt.subplots(figsize=(fig_width, fig_height))

    sns.barplot(x=cluster_counts.index.astype(str), y=cluster_counts.values, ax=current_ax, palette="viridis")
    current_ax.set_title(plot_title, fontsize=15)
    current_ax.set_xlabel("Cluster ID", fontsize=12)
    current_ax.set_ylabel("Number of Records", fontsize=12)
    
    if len(cluster_counts.index) > 15 :
        current_ax.tick_params(axis='x', rotation=90, labelsize=8)
    elif len(cluster_counts.index) > 5:
        current_ax.tick_params(axis='x', rotation=45, ha='right', labelsize=10)
    else:
        current_ax.tick_params(axis='x', rotation=0, labelsize=10)

    for i, v in enumerate(cluster_counts.values):
        current_ax.text(i, v + (cluster_counts.values.max() * 0.015), str(v), color='black', ha='center', va='bottom', fontsize=9)
    
    current_ax.set_ylim(top=cluster_counts.values.max() * 1.1)

    if ax is None:
        plt.tight_layout()
        plt.show()

def plot_noise_vs_clustered_summary(
    df: pd.DataFrame,
    cluster_column: str,
    domain_name: Optional[str] = None,
    ax: Optional[plt.Axes] = None
) -> None:
    """
    Generates a bar plot showing the count of noise records vs. actual clustered records
    for a specified domain.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        cluster_column (str): The name of the column containing cluster labels.
        domain_name (Optional[str]): The name of the domain to filter by.
                                     If None, data is not filtered by domain.
                                     The domain_name column in df must be boolean.
        ax (Optional[plt.Axes]): A matplotlib axes object to plot on.
                                 If None, a new figure and axes will be created.
    """
    if cluster_column not in df.columns:
        print(f"Error: Cluster column '{cluster_column}' not found in DataFrame.")
        return

    df_to_plot = df.copy()
    plot_title = "Noise vs. Clustered Records"
    original_total_records = len(df_to_plot)

    if domain_name:
        if domain_name not in df.columns:
            print(f"Warning: Domain column '{domain_name}' not found in DataFrame. Plotting for all data in df.")
        elif not pd.api.types.is_bool_dtype(df_to_plot[domain_name]):
            print(f"Warning: Domain column '{domain_name}' is not boolean. Plotting for all data in df.")
        else:
            df_to_plot = df_to_plot[df_to_plot[domain_name] == True]
            plot_title = f"Noise vs. Clustered Records in {domain_name}"
            original_total_records = len(df_to_plot) # Update total for the specific domain
            
    if df_to_plot.empty:
        if domain_name:
            print(f"No data to plot for domain '{domain_name}'.")
        else:
            print("No data to plot (DataFrame is empty).")
        return

    noise_count = (df_to_plot[cluster_column] == -1).sum()
    clustered_count = (df_to_plot[cluster_column] != -1).sum()
    
    total_processed_records = noise_count + clustered_count

    if total_processed_records == 0:
        print(f"No records (noise or clustered) found for the specified selection.")
        return

    labels = ['Noise Records (-1)', 'Clustered Records (>=0)']
    counts = [noise_count, clustered_count]
    
    # Calculate percentages
    percentages = [(count / total_processed_records * 100) if total_processed_records > 0 else 0 for count in counts]

    current_ax = ax
    if current_ax is None:
        fig, current_ax = plt.subplots(figsize=(8, 6))

    bars = sns.barplot(x=labels, y=counts, ax=current_ax, palette=["#FF6347", "#4682B4"]) # Tomato for noise, SteelBlue for clustered
    current_ax.set_title(plot_title, fontsize=15)
    current_ax.set_ylabel("Number of Records", fontsize=12)

    for i, bar in enumerate(bars.patches):
        count_val = counts[i]
        percentage_val = percentages[i]
        current_ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + (max(counts) * 0.015), # Position text above bar
            f"{count_val}\n({percentage_val:.1f}%)", # Show count and percentage
            ha='center',
            va='bottom',
            fontsize=10
        )
    
    current_ax.set_ylim(top=max(counts) * 1.15) # Adjust ylim to make space for text

    print(f"Analysis for {domain_name if domain_name else 'all data'}:")
    print(f"  Total records processed: {total_processed_records}")
    print(f"  Noise records (-1): {noise_count} ({percentages[0]:.1f}%)")
    print(f"  Clustered records (>=0): {clustered_count} ({percentages[1]:.1f}%)")


    if ax is None:
        plt.tight_layout()
        plt.show()

In [ ]:
def plot_cluster_size_distribution(
    df: pd.DataFrame,
    cluster_column: str,
    domain_name: Optional[str] = None,
    bin_step: int = 20,
    max_bins: Optional[int] = 10,
    ax: Optional[plt.Axes] = None
) -> None:
    """
    Generates a bar plot showing the distribution of cluster sizes.
    It counts how many clusters fall into predefined size bins.
    Includes a text box with min, max, and median actual cluster sizes.
    Bars are colored blue.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        cluster_column (str): The name of the column containing cluster labels.
        domain_name (Optional[str]): The name of the domain to filter by.
                                     If None, data is not filtered by domain.
                                     The domain_name column in df must be boolean.
        bin_step (int): The step size for creating bins (e.g., 20 means bins like 0-19, 20-39).
        max_bins (Optional[int]): Approximate maximum number of bins to create.
                                  The last bin will be an "X+" bin if sizes exceed this.
                                  If None, bins will go up to the max cluster size.
        ax (Optional[plt.Axes]): A matplotlib axes object to plot on.
                                 If None, a new figure and axes will be created.
    """
    if cluster_column not in df.columns:
        print(f"Error: Cluster column '{cluster_column}' not found in DataFrame.")
        return

    df_to_analyze = df.copy()
    plot_title = "Distribution of Cluster Sizes"

    if domain_name:
        if domain_name not in df.columns:
            print(f"Warning: Domain column '{domain_name}' not found in DataFrame. Analyzing all data in df.")
        elif not pd.api.types.is_bool_dtype(df_to_analyze[domain_name]):
            print(f"Warning: Domain column '{domain_name}' is not boolean. Analyzing for all data in df.")
        else:
            df_to_analyze = df_to_analyze[df_to_analyze[domain_name] == True]
            plot_title = f"Distribution of Cluster Sizes in {domain_name}"

    if df_to_analyze.empty:
        if domain_name:
            print(f"No data to analyze for domain '{domain_name}'.")
        else:
            print("No data to analyze (DataFrame is empty).")
        return

    # Get sizes of actual clusters (excluding noise -1)
    cluster_sizes = df_to_analyze[df_to_analyze[cluster_column] != -1][cluster_column].value_counts()

    if cluster_sizes.empty:
        print(f"No actual (non-noise) clusters found for '{domain_name if domain_name else 'all data'}'.")
        return

    min_actual_cluster_size = cluster_sizes.min()
    max_actual_cluster_size = cluster_sizes.max()
    median_actual_cluster_size = cluster_sizes.median() # Calculate median

    # Define bins
    max_size_for_bins = cluster_sizes.max()
    
    bins = []
    if max_bins is not None and (max_size_for_bins / bin_step) > (max_bins - 1):
        upper_limit_regular_bins = (max_bins - 1) * bin_step
        bins = list(np.arange(0, upper_limit_regular_bins + bin_step, bin_step))
        
        if bins[-1] <= max_size_for_bins:
            bins.append(np.inf)
        else: 
            bins = [b for b in bins if b < max_size_for_bins] 
            if not bins or bins[-1] < max_size_for_bins : 
                 bins.append(max_size_for_bins +1) 
            bins.append(np.inf) 
    else:
        bins = list(np.arange(0, max_size_for_bins + bin_step, bin_step))

    final_bins = [0]
    for b_val in bins:
        if b_val > final_bins[-1]: 
            final_bins.append(b_val)
    
    if final_bins[-1] != np.inf:
        if max_bins is not None and len(final_bins) >= max_bins: 
            final_bins[-1] = np.inf
        elif final_bins[-1] <= max_size_for_bins : 
            final_bins.append(np.inf)

    bins = sorted(list(set(final_bins)))
    if not bins: bins = [0, np.inf] 
    if bins[0] != 0 and 0 not in bins: bins.insert(0,0)
    bins = sorted(list(set(bins))) 

    if max_size_for_bins < bin_step and len(bins) > 2 and bins[1] != np.inf :
        bins = [0, max_size_for_bins + 1, np.inf] 
        bins = sorted(list(set(bins)))


    bin_labels = []
    if len(bins) < 2: 
        print("Warning: Not enough bins to create labels for cluster size distribution.")
        if cluster_sizes.shape[0] > 0:
             print(f"All {cluster_sizes.shape[0]} clusters have sizes: {cluster_sizes.values}")
        return

    for i in range(len(bins) - 1):
        lower_bound = int(bins[i])
        upper_bound = bins[i+1]

        if lower_bound == 0: 
            actual_lower_bound_display = 1 
        else:
            actual_lower_bound_display = lower_bound
        
        if upper_bound == np.inf:
            bin_labels.append(f"{actual_lower_bound_display}+ records")
        else:
            bin_labels.append(f"{actual_lower_bound_display}-{int(upper_bound-1)} records")
    
    binned_cluster_sizes = pd.cut(cluster_sizes, bins=bins, labels=bin_labels, right=False, include_lowest=True)
    
    distribution = binned_cluster_sizes.value_counts().sort_index()

    if distribution.empty:
        print("Could not generate cluster size distribution (no data fell into bins).")
        print(f"Cluster sizes (min: {min_actual_cluster_size}, median: {median_actual_cluster_size}, max: {max_actual_cluster_size}): {cluster_sizes.values if not cluster_sizes.empty else 'empty'}")
        print(f"Bins used: {bins}")
        print(f"Bin labels: {bin_labels}")
        return

    current_ax = ax
    if current_ax is None:
        fig_width = max(10, len(distribution) * 0.9) 
        fig, current_ax = plt.subplots(figsize=(fig_width, 7)) 

    sns.barplot(x=distribution.index, y=distribution.values, ax=current_ax, color="steelblue")
    
    current_ax.set_title(plot_title, fontsize=15)
    current_ax.set_xlabel("Cluster Size (Number of Records in Cluster)", fontsize=12)
    current_ax.set_ylabel("Number of Clusters", fontsize=12)
    
    current_ax.tick_params(axis='x', rotation=45, labelsize=10) 

    for i, v in enumerate(distribution.values):
        current_ax.text(i, v + (distribution.values.max() * 0.015), str(v), color='black', ha='center', va='bottom', fontsize=9)
    
    current_ax.set_ylim(top=distribution.values.max() * 1.15) 

    # Add text box for min/median/max cluster size
    textstr = (f'Min Cluster Size: {min_actual_cluster_size}\n'
               f'Median Cluster Size: {median_actual_cluster_size:.0f}\n' # Format median as integer
               f'Max Cluster Size: {max_actual_cluster_size}')
    props = dict(boxstyle='round,pad=0.5', facecolor='wheat', alpha=0.7)
    current_ax.text(0.97, 0.97, textstr, transform=current_ax.transAxes, fontsize=10,
                    verticalalignment='top', horizontalalignment='right', bbox=props)

    if ax is None:
        plt.tight_layout()
        plt.show()


#### Domains

In [ ]:
def plot_monthly_publications_per_domain(
    df: pd.DataFrame,
    domain_column_names: List[str],
    date_column: str = 'first_date',
    figsize_per_subplot: Tuple[int, int] = (14, 4)
):
    """
    Creates a series of bar plots, one for each domain, showing the number of
    papers published each month.

    Args:
        df (pd.DataFrame): The DataFrame containing publication data.
        domain_column_names (List[str]): A list of column names, where each column
                                         represents a domain and contains boolean values
                                         (True if the paper belongs to the domain).
        date_column (str): The name of the column containing publication dates.
        figsize_per_subplot (Tuple[int, int]): Tuple specifying (width, height) for each subplot.
    """

    # Ensure the date column is in datetime format for proper processing
    # Operate on a copy to avoid modifying the original DataFrame passed to the function
    df_processed = df.copy()
    if not pd.api.types.is_datetime64_any_dtype(df_processed[date_column]):
        try:
            df_processed[date_column] = pd.to_datetime(df_processed[date_column])
            print(f"  Info: Converted '{date_column}' to datetime objects for plotting.")
        except Exception as e:
            print(f"  Error: Could not convert '{date_column}' to datetime: {e}. Aborting plot generation.")
            return

    # Create a 'YearMonth' column for grouping
    df_processed['YearMonthPlotting'] = df_processed[date_column].dt.to_period('M')

    num_domains = len(domain_column_names)
    if num_domains == 0:
        print("  No domain columns provided. Nothing to plot.")
        return

    ncols = 2
    nrows = (num_domains + ncols - 1) // ncols

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                             figsize=(figsize_per_subplot[0] * ncols, figsize_per_subplot[1] * nrows),
                             squeeze=False) # squeeze=False ensures axes is always a 2D array
    axes = axes.ravel() # Flatten to 1D array for easy iteration

    plot_count = 0
    for i, domain_col in enumerate(domain_column_names):
        ax = axes[i] # Get the current axis

        # Filter data for the current domain
        df_domain = df_processed[df_processed[domain_col] == True]

        monthly_counts = df_domain.groupby('YearMonthPlotting').size()

        # Convert PeriodIndex to Timestamps for better plotting control with matplotlib
        plot_index = monthly_counts.index.to_timestamp()

        ax.bar(plot_index, monthly_counts.values, width=20) # width in days for monthly data
        ax.set_title(f'Monthly Publications: {domain_col}', fontsize=14)
        ax.set_ylabel('Number of Papers', fontsize=10)
        ax.set_xlabel('Month', fontsize=10)
        ax.grid(axis='y', linestyle='--', alpha=0.7)

        # Improve x-axis tick labels for readability
        num_months_on_axis = len(plot_index)
        if num_months_on_axis > 12: # If more than 1 year of data
            # Show major ticks for years, minor for months, format as YYYY-MM
            ax.xaxis.set_major_locator(mdates.YearLocator())
            ax.xaxis.set_minor_locator(mdates.MonthLocator((1,4,7,10))) # Every 3 months for minor if too dense
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
            plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
        else: # For shorter periods, default formatting might be okay or just rotate
            plt.setp(ax.get_xticklabels(), rotation=30, ha='right')


        plot_count += 1

    # Turn off any unused subplots if the number of domains is not a perfect fit for the grid
    for j in range(plot_count, nrows * ncols):
        fig.delaxes(axes[j])

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout, rect leaves space for suptitle and bottom
    fig.suptitle('Monthly Publication Counts per Domain', fontsize=18, y=0.99)
    plt.show()

domain_cols_for_plot = [
    "Physics", "Computer Science", "Statistics", "Mathematics",
    "Electrical Engineering and Systems Science", "Quantitative Biology",
    "Economics", "Quantitative Finance"
]

plot_monthly_publications_per_domain(df, domain_cols_for_plot)

In [ ]:
def plot_total_publications_per_domain(
    df: pd.DataFrame,
    domain_column_names: List[str],
    figsize: Tuple[int, int] = (12, 7)
):
    """
    Creates a single bar plot showing the total number of publications for each domain,
    with x-axis tick labels wrapped if they are too long.

    Args:
        df (pd.DataFrame): The DataFrame containing publication data.
        domain_column_names (List[str]): A list of column names, where each column
                                         represents a domain and contains boolean values
                                         (True if the paper belongs to the domain).
        figsize (Tuple[int, int]): Tuple specifying (width, height) for the plot.
    """

    if not domain_column_names:
        print("  No domain columns provided. Nothing to plot.")
        return

    domain_counts = {}
    for domain_col in domain_column_names:
        if domain_col not in df.columns:
            print(f"  Warning: Domain column '{domain_col}' not found in DataFrame. Skipping.")
            continue
        # Assuming boolean True means the paper belongs to the domain
        domain_counts[domain_col] = df[df[domain_col] == True].shape[0]

    if not domain_counts:
        print("  No valid domain columns found or no data for specified domains. Nothing to plot.")
        return

    sorted_domain_counts = dict(sorted(domain_counts.items(), key=lambda item: item[1], reverse=True))

    domains = list(sorted_domain_counts.keys())
    counts = list(sorted_domain_counts.values())

    plt.figure(figsize=figsize)
    bars = plt.bar(domains, counts, color='#4169e1')

    # Add counts on top of bars
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.01 * max(counts),
                 int(yval), ha='center', va='bottom', fontsize=9)


    plt.title('Total Publications per Domain', fontsize=16)
    plt.xlabel('', fontsize=12)
    plt.ylabel('Total Number of Publications', fontsize=12)

    # Wrap x-axis tick labels
    # You can adjust the width parameter as needed
    wrapped_labels = [textwrap.fill(label, width=15) for label in domains]
    plt.xticks(ticks=range(len(domains)), labels=wrapped_labels, rotation=45, ha='right', fontsize=10)

    plt.yticks(fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


plot_total_publications_per_domain(df, domain_cols_for_plot)

##### Physics

In [ ]:
domain_to_process = 'Physics'

prep_result = prepare_domain_data(
    domain_to_process, df, metadata_features, keyword_cols_created, domain_top_keywords_map
)

df_domain, X_domain_dense, relevant_keywords = prep_result

In [ ]:
# Parameters for PCA pre-processing
PCA_ACTIVATION_THRESHOLD = 150  # Apply PCA if number of features exceeds this
PCA_TARGET_COMPONENTS = 100     # Target number of components after PCA


# Ensure X_domain_dense is valid before proceeding
if X_domain_dense is not None and X_domain_dense.shape[0] > 0:
    print(f"\n--- PCA Pre-processing for domain: {domain_to_process} ---")
    print(f"Original X_domain_dense shape: {X_domain_dense.shape}")


    data_for_umap = X_domain_dense  # Default to original data


    if X_domain_dense.shape[1] > PCA_ACTIVATION_THRESHOLD:
        print(f"Number of features ({X_domain_dense.shape[1]}) > {PCA_ACTIVATION_THRESHOLD}. Applying PCA.")


        # 1. Scale data before PCA
        pca_input_scaler = StandardScaler()
        X_domain_scaled_for_pca = pca_input_scaler.fit_transform(X_domain_dense)
        print("  Data scaled for PCA.")


        # 2. Determine the number of PCA components
        # n_components must be <= min(n_samples, n_features) and > 0
        # It should also be less than the original number of features to ensure reduction.
        _n_samples = X_domain_scaled_for_pca.shape[0]
        _n_features = X_domain_scaled_for_pca.shape[1]


        # Calculate a valid number of components for PCA
        # Target PCA_TARGET_COMPONENTS, but ensure it's less than current features and samples.
        if _n_features <= 1 or _n_samples <=1 : # PCA not meaningful or possible
            n_pca_components = 0
        else:
            # Ensure reduction: n_components should be < _n_features
            # Ensure validity: n_components should be <= _n_samples
            # Ensure target: n_components aims for PCA_TARGET_COMPONENTS
            n_pca_components = min(PCA_TARGET_COMPONENTS, _n_samples, _n_features -1)


        if n_pca_components > 0:
            print(f"  Applying PCA with n_components = {n_pca_components}")
            # Assuming RANDOM_STATE is defined in your notebook (e.g., RANDOM_STATE = 42)
            pca_reducer = PCA(n_components=n_pca_components, random_state=RANDOM_STATE)
            X_domain_intermediate_pca = pca_reducer.fit_transform(X_domain_scaled_for_pca)
            
            print(f"  Shape after PCA: {X_domain_intermediate_pca.shape}")
            if hasattr(pca_reducer, 'explained_variance_ratio_'):
                print(f"  Total explained variance by {pca_reducer.n_components_} PCA components: {np.sum(pca_reducer.explained_variance_ratio_):.4f}")
            data_for_umap = X_domain_intermediate_pca  # Use PCA output for UMAP
        else:
            print(f"  PCA not applied: Calculated n_pca_components ({n_pca_components}) is not suitable for reduction. Using original features.")
            # data_for_umap remains X_domain_dense (the original features)
    else:
        print(f"Number of features ({X_domain_dense.shape[1]}) is not greater than {PCA_ACTIVATION_THRESHOLD}. PCA not applied.")
        # data_for_umap remains X_domain_dense
    
    print(f"--- End of PCA Pre-processing. Data shape for UMAP: {data_for_umap.shape if data_for_umap is not None else 'None'} ---")


else:
    print(f"X_domain_dense for domain '{domain_to_process}' is None or empty. Skipping PCA and UMAP.")
    data_for_umap = None # Ensure data_for_umap is defined for the next step, even if None


In [ ]:
if data_for_umap is not None:
    reduction_result = reduce_dimensionality(data_for_umap, umap_params={
        'n_components': 10,
        'n_neighbors': 50,
        'min_dist': 0.0,
        'metric': 'euclidean',
        'random_state': RANDOM_STATE,
        'verbose': True,
        'low_memory': True
    })
else:
    reduction_result = None


# Check reduction_result before unpacking
if reduction_result:
    X_domain_reduced_umap, scaler, reducer = reduction_result
else:
    print(f"Dimensionality reduction (UMAP) failed for domain {domain_to_process} or was skipped.")
    # Initialize to prevent errors if these variables are expected later,
    # though subsequent clustering steps should also check if X_domain_reduced_umap is valid.
    X_domain_reduced_umap = None
    scaler = None
    reducer = None

In [ ]:
import joblib

if X_domain_reduced_umap is not None:
    try:
        joblib.dump(X_domain_reduced_umap, 'X_domain_reduced_umap_physics.joblib')
        print("X_domain_reduced_umap_physics.joblib saved successfully.")
    except Exception as e:
        print(f"Error saving X_domain_reduced_umap_physics.joblib: {e}")
else:
    print("X_domain_reduced_umap is None, skipping save.")


if scaler is not None:
    try:
        joblib.dump(scaler, 'scaler_physics.joblib')
        print("scaler_physics.joblib saved successfully.")
    except Exception as e:
        print(f"Error saving scaler_physics.joblib: {e}")
else:
    print("scaler is None, skipping save.")


if reducer is not None:
    try:
        joblib.dump(reducer, 'reducer_physics.joblib')
        print("reducer_physics.joblib saved successfully.")
    except Exception as e:
        print(f"Error saving reducer_physics.joblib: {e}")
else:
    print("reducer is None, skipping save.")


In [ ]:
import joblib
# Attempt to reload the UMAP reduced data
try:
    X_domain_reduced_umap = joblib.load('X_domain_reduced_umap_physics.joblib')
    print("X_domain_reduced_umap_physics.joblib reloaded successfully.")
except FileNotFoundError:
    print("Error: X_domain_reduced_umap_physics.joblib not found.")
    X_domain_reduced_umap = None
except Exception as e:
    print(f"Error reloading X_domain_reduced_umap_physics.joblib: {e}")
    X_domain_reduced_umap = None


# Attempt to reload the scaler
try:
    scaler = joblib.load('scaler_physics.joblib')
    print("scaler_physics.joblib reloaded successfully.")
except FileNotFoundError:
    print("Error: scaler_physics.joblib not found.")
    scaler = None
except Exception as e:
    print(f"Error reloading scaler_physics.joblib: {e}")
    scaler = None


# Attempt to reload the reducer
try:
    reducer = joblib.load('reducer_physics.joblib')
    print("reducer_physics.joblib reloaded successfully.")
except FileNotFoundError:
    print("Error: reducer_physics.joblib not found.")
    reducer = None
except Exception as e:
    print(f"Error reloading reducer_physics.joblib: {e}")
    reducer = None

In [ ]:
# sample_size = 100000
# X_domain_dense_sampled = X_domain_dense[np.random.choice(X_domain_dense.shape[0], sample_size, replace=False)]

# reduction_result = reduce_dimensionality(X_domain_dense_sampled, umap_params={
#     'n_components': 10,
#     'n_neighbors': 50,
#     'min_dist': 0.0,
#     'metric': 'euclidean',
#     'random_state': RANDOM_STATE,
#     'verbose': True,
#     'low_memory': True
# })

reduction_result = reduce_dimensionality(X_domain_dense, umap_params={
    'n_components': 10,
    'n_neighbors': 50,
    'min_dist': 0.0,
    'metric': 'euclidean',
    'random_state': RANDOM_STATE,
    'verbose': True,
    'low_memory': True
})

X_domain_reduced_umap, scaler, reducer = reduction_result

In [ ]:
# --- HDBSCAN Clustering  ---
# Adjust min_cluster_size and min_samples based on your domain's data size and density
# Smaller min_cluster_size allows for smaller, more granular clusters.
# Larger min_samples makes clustering more conservative (more points become noise).
hdbscan_params = {
    "min_cluster_size": 20,  # Example: minimum number of papers to form a cluster
    "min_samples": 10,        # Example: lower values make it easier to form clusters
    "metric": 'euclidean',    # Metric used for UMAP output
    "cluster_selection_method": 'eom' # 'eom' or 'leaf'
}

clustering_result_hdbscan = apply_hdbscan_clustering(
    X_domain_reduced_umap,
    df, # Main DataFrame
    df_domain.index, # Index for the current domain's data in the main df
    domain_to_process,
    **hdbscan_params
)

In [ ]:
# df is modified in-place by apply_hdbscan_clustering
_, hdbscan_model, n_clusters_found_hdbscan = clustering_result_hdbscan

chosen_c_for_analysis = n_clusters_found_hdbscan
print(f"HDBSCAN identified {chosen_c_for_analysis} clusters for analysis (excluding noise).")

In [ ]:
# --- Analyze Sub-Clusters (mostly the same function call) ---
# The analyze_sub_clusters function should handle the -1 label for noise if it appears in plots.
# It iterates over unique cluster labels found.
analysis_output = analyze_sub_clusters(
    df,  # df already updated with HDBSCAN labels
    domain_to_process,
    chosen_c_for_analysis, # Number of actual clusters (for print statements, etc.)
    relevant_keywords, # Features used for UMAP
    metadata_features,
    rolling_window_param=ROLLING_WINDOW
)

smoothed_proportions_df = None
top_keywords_data_for_domain = {}

smoothed_proportions_df, top_keywords_data_for_domain = analysis_output

# --- Store Results ---
domain_cluster_results = {}

domain_cluster_results[domain_to_process] = {
    'k_found': chosen_c_for_analysis,
    'hdbscan_model': hdbscan_model,
    'umap_reducer': reducer,
    'scaler': scaler,
    'features_used': relevant_keywords,
    'hdbscan_params': hdbscan_params
}

domain_proportions_data = {}
domain_top_keywords_info = {}

domain_proportions_data[domain_to_process] = smoothed_proportions_df
domain_top_keywords_info[domain_to_process] = top_keywords_data_for_domain

In [ ]:
plot_noise_vs_clustered_summary(df=df,
                                cluster_column='sub_cluster',
                                domain_name=domain_to_process)

plot_cluster_record_counts(df=df,
                           cluster_column='sub_cluster',
                           domain_name=domain_to_process,
                           top_n=50
                           )

plot_cluster_size_distribution(df=df,
                               cluster_column='sub_cluster',
                               domain_name=domain_to_process,
                               bin_step=10,      
                               max_bins=10)    

In [ ]:
emerging_ids_qb = identify_emerging_cluster_ids(
    df,
    domain_name=domain_to_process,
    cluster_column='sub_cluster',
    date_column='first_date',
    recent_months_window=4,
    min_papers_recent_period=2,        # Example: cluster needs at least 5 papers recently
    emerging_ratio_threshold=1,      # Example: recent proportion must be 1.5x baseline
    emerging_diff_threshold=0.01,      # Example: recent proportion must be at least 1% higher
    newly_active_min_recent_prop=0.0001 # Example: new cluster must be at least 0.5% of recent papers
)

In [ ]:
clusters_to_combine = list(range(chosen_c_for_analysis))
colors_for_combine = COLORS_2.copy()

clusters_to_preserve_color = emerging_ids_qb

for cluster_id in colors_for_combine:
    if cluster_id not in clusters_to_preserve_color:
        colors_for_combine[cluster_id] = EXCLUSION_COLOR 

proportions_df_to_plot = domain_proportions_data[domain_to_process]
top_keywords_for_plot = domain_top_keywords_info[domain_to_process]

plot_combined_trends(
    proportions_df_to_plot,
    clusters_to_combine,
    colors_for_combine,
    top_keywords_for_plot,
    domain_to_process,
    rolling_window=ROLLING_WINDOW,
    keyword_exclusion_color=EXCLUSION_COLOR,
    y_axis_scale = 'linear'
)

In [ ]:
cluster = 218

keywords_for_this_cluster = domain_top_keywords_info[domain_to_process][cluster]

plot_cluster_keyword_trends_plotly(
    df=df,
    domain_name=domain_to_process,
    cluster_id=cluster,
    all_cluster_keywords=keywords_for_this_cluster,
    num_keywords_to_plot=10,
    custom_colors=COLORS,
    rolling_window=12 
)

##### Computer Science

##### Statistics

##### Mathematics

##### Electrical Engineering and Systems Science

##### Quantitative Biology

###### K-Means

In [ ]:
domain_to_process = 'Quantitative Biology'

prep_result = prepare_domain_data(domain_to_process, df, metadata_features, keyword_cols_created, domain_top_keywords_map)
 
df_domain, X_domain_dense, relevant_keywords = prep_result

reduction_result = reduce_dimensionality(X_domain_dense, umap_params={
    'n_components': 50,
    'n_neighbors': 20,
    'min_dist': 0.1,
    'metric': 'euclidean',
    'random_state': 42,
    'verbose': True
})

X_domain_reduced_umap, scaler, reducer = reduction_result

In [ ]:
suggested_k = evaluate_k(X_domain_reduced_umap, domain_to_process,
                         k_range=DEFAULT_K_RANGE, sample_size=DEFAULT_SILHOUETTE_SAMPLE_SIZE,
                         kmeans_params=DEFAULT_KMEANS_PARAMS)

In [ ]:
chosen_k = 5

clustering_result = apply_clustering(X_domain_reduced_umap, chosen_k, df, df_domain.index,
                                    domain_to_process, kmeans_params=DEFAULT_KMEANS_PARAMS)

if clustering_result:
    _, kmeans_model = clustering_result

    analysis_output = analyze_sub_clusters(
        df, domain_to_process, chosen_k, relevant_keywords, metadata_features
    )

    smoothed_proportions_df = None
    top_keywords_data_for_domain = {} # Initialize as empty dict

    if analysis_output is not None:
        smoothed_proportions_df, top_keywords_data_for_domain = analysis_output

    domain_cluster_results = {}

    domain_cluster_results[domain_to_process] = {
    'k': chosen_k,
    'kmeans_model': kmeans_model,
    'umap_reducer': reducer,
    'scaler': scaler,
    'features_used': metadata_features + relevant_keywords
    }

    # --- Store Proportions Data and Keywords Data Globally ---
    if 'domain_proportions_data' not in locals():
         domain_proportions_data = {}
    if 'domain_top_keywords_info' not in locals(): # New dictionary for keywords
         domain_top_keywords_info = {}

    if smoothed_proportions_df is not None:
        domain_proportions_data[domain_to_process] = smoothed_proportions_df
        print(f"\nSmoothed proportions data for '{domain_to_process}' stored.")
    else:
        domain_proportions_data[domain_to_process] = None
        print(f"\nNo proportions data stored for '{domain_to_process}'.")

    if top_keywords_data_for_domain: # Check if it's not empty
        domain_top_keywords_info[domain_to_process] = top_keywords_data_for_domain
        print(f"Top keywords data for '{domain_to_process}' stored.")
    else:
        domain_top_keywords_info[domain_to_process] = {} # Store empty dict if no keywords
        print(f"No top keywords data stored for '{domain_to_process}'.")

    print(f"\n--- Finished analysis block for domain: {domain_to_process} ---")
else:
    print(f"Clustering failed for domain {domain_to_process}, skipping analysis.")

In [ ]:
clusters_to_combine = list(range(chosen_k))
colors_for_combine = COLORS.copy()

clusters_to_preserve_color = [3, 4]

for cluster_id in colors_for_combine:
    if cluster_id not in clusters_to_preserve_color:
        colors_for_combine[cluster_id] = EXCLUSION_COLOR 

proportions_df_to_plot = domain_proportions_data[domain_to_process]
top_keywords_for_plot = domain_top_keywords_info[domain_to_process]

plot_combined_trends(
    proportions_df_to_plot,
    clusters_to_combine,
    colors_for_combine,
    top_keywords_for_plot,
    domain_to_process,
    rolling_window=ROLLING_WINDOW,
    keyword_exclusion_color=EXCLUSION_COLOR,
    y_axis_scale = 'linear'
)

# plot_combined_trends(
#     proportions_df_to_plot,
#     clusters_to_combine,
#     colors_for_combine,
#     top_keywords_for_plot,
#     domain_to_process,
#     rolling_window=ROLLING_WINDOW,
#     keyword_exclusion_color=EXCLUSION_COLOR,
#     y_axis_scale = 'log'
# )

In [ ]:
cluster = 4

keywords_for_this_cluster = domain_top_keywords_info[domain_to_process][cluster]

plot_cluster_keyword_trends_plotly(
    df=df,
    domain_name=domain_to_process,
    cluster_id=cluster,
    all_cluster_keywords=keywords_for_this_cluster,
    num_keywords_to_plot=10,
    custom_colors=COLORS,
    rolling_window=12 
)

###### HDBSCAN

In [ ]:
domain_to_process = 'Quantitative Biology'

prep_result = prepare_domain_data(
    domain_to_process, df, metadata_features, keyword_cols_created, domain_top_keywords_map
)

df_domain, X_domain_dense, relevant_keywords = prep_result

reduction_result = reduce_dimensionality(X_domain_dense, umap_params={
    'n_components': 50,
    'n_neighbors': 20,
    'min_dist': 0.0,
    'metric': 'euclidean',
    'random_state': RANDOM_STATE,
    'verbose': True
})

X_domain_reduced_umap, scaler, reducer = reduction_result

# --- HDBSCAN Clustering  ---
# Adjust min_cluster_size and min_samples based on your domain's data size and density
# Smaller min_cluster_size allows for smaller, more granular clusters.
# Larger min_samples makes clustering more conservative (more points become noise).
hdbscan_params = {
    "min_cluster_size": 20,  # Example: minimum number of papers to form a cluster
    "min_samples": 10,        # Example: lower values make it easier to form clusters
    "metric": 'euclidean',    # Metric used for UMAP output
    "cluster_selection_method": 'eom' # 'eom' or 'leaf'
}

clustering_result_hdbscan = apply_hdbscan_clustering(
    X_domain_reduced_umap,
    df, # Main DataFrame
    df_domain.index, # Index for the current domain's data in the main df
    domain_to_process,
    **hdbscan_params
)

# df is modified in-place by apply_hdbscan_clustering
_, hdbscan_model, n_clusters_found_hdbscan = clustering_result_hdbscan

chosen_c_for_analysis = n_clusters_found_hdbscan
print(f"HDBSCAN identified {chosen_c_for_analysis} clusters for analysis (excluding noise).")

In [ ]:
# --- Analyze Sub-Clusters (mostly the same function call) ---
# The analyze_sub_clusters function should handle the -1 label for noise if it appears in plots.
# It iterates over unique cluster labels found.
analysis_output = analyze_sub_clusters(
    df,  # df already updated with HDBSCAN labels
    domain_to_process,
    chosen_c_for_analysis, # Number of actual clusters (for print statements, etc.)
    relevant_keywords, # Features used for UMAP
    metadata_features,
    rolling_window_param=ROLLING_WINDOW
)

smoothed_proportions_df = None
top_keywords_data_for_domain = {}

smoothed_proportions_df, top_keywords_data_for_domain = analysis_output

# --- Store Results ---
domain_cluster_results = {}

domain_cluster_results[domain_to_process] = {
    'k_found': chosen_c_for_analysis,
    'hdbscan_model': hdbscan_model,
    'umap_reducer': reducer,
    'scaler': scaler,
    'features_used': relevant_keywords,
    'hdbscan_params': hdbscan_params
}

domain_proportions_data = {}
domain_top_keywords_info = {}

domain_proportions_data[domain_to_process] = smoothed_proportions_df
domain_top_keywords_info[domain_to_process] = top_keywords_data_for_domain

In [ ]:
plot_noise_vs_clustered_summary(df=df,
                                cluster_column='sub_cluster',
                                domain_name=domain_to_process)

plot_cluster_record_counts(df=df,
                           cluster_column='sub_cluster',
                           domain_name=domain_to_process,
                           top_n=50
                           )

plot_cluster_size_distribution(df=df,
                               cluster_column='sub_cluster',
                               domain_name=domain_to_process,
                               bin_step=10,      
                               max_bins=10)      


In [ ]:
emerging_ids_qb = identify_emerging_cluster_ids(
    df,
    domain_name=domain_to_process,
    cluster_column='sub_cluster',
    date_column='first_date',
    recent_months_window=4,
    min_papers_recent_period=2,        # Example: cluster needs at least 5 papers recently
    emerging_ratio_threshold=1,      # Example: recent proportion must be 1.5x baseline
    emerging_diff_threshold=0.01,      # Example: recent proportion must be at least 1% higher
    newly_active_min_recent_prop=0.0001 # Example: new cluster must be at least 0.5% of recent papers
)

In [ ]:
clusters_to_combine = list(range(chosen_c_for_analysis))
colors_for_combine = COLORS_2.copy()

clusters_to_preserve_color = emerging_ids_qb

for cluster_id in colors_for_combine:
    if cluster_id not in clusters_to_preserve_color:
        colors_for_combine[cluster_id] = EXCLUSION_COLOR 

proportions_df_to_plot = domain_proportions_data[domain_to_process]
top_keywords_for_plot = domain_top_keywords_info[domain_to_process]

plot_combined_trends(
    proportions_df_to_plot,
    clusters_to_combine,
    colors_for_combine,
    top_keywords_for_plot,
    domain_to_process,
    rolling_window=ROLLING_WINDOW,
    keyword_exclusion_color=EXCLUSION_COLOR,
    y_axis_scale = 'linear'
)

In [ ]:
cluster = 218

keywords_for_this_cluster = domain_top_keywords_info[domain_to_process][cluster]

plot_cluster_keyword_trends_plotly(
    df=df,
    domain_name=domain_to_process,
    cluster_id=cluster,
    all_cluster_keywords=keywords_for_this_cluster,
    num_keywords_to_plot=10,
    custom_colors=COLORS,
    rolling_window=12 
)

##### Economics

##### Quantitative Finance